## <a name="index"></a> Indice

[¿Que es un Data Warehouse?](#mark_01)

[Bases de datos columnares y arquitectura orientada a optimización de consultas](#mark_02)

[¿Cómo funciona AWS Redshift?](#mark_03)

[Creando nuestro entorno de trabajo en AWS](#mark_04)

[Cómo conectarnos a Redshift](#mark_05)

[Creación de tablas en Redshift](#mark_06)

[Carga de datos en S3](#mark_07)

[Ingesta S3 --> Redshift](#mark_08)

[Algoritmos de compresión con Redshift 1° Parte.](#mark_09)

   - [RAW - Sin compresión](#mark_09.0)
   
   - [AZ64](#mark_09.1)
   
   - [Codificación por diccionarios de bytes](#mark_09.2)
   
   - [Codificacion Delta](#mark_09.3)
   
   - [LZO](#mark_09.4)

[Algoritmos de compresión con Redshift 2° Parte.](#mark_10)

   - [Codificación Mostly](#mark_10.0)
   
   - [Codificación de run length](#mark_10.1)
   
   - [Codificaciones Text255 y Text32k](#mark_10.2)
   
   - [Codificación ZSTD](#mark_10.3)
   

[Codificando/Compresión de Columnas](#mark_11)

[Análisis de desempeño con diferentes tipos de compresión](#mark_12)

   - [STV_TBL_PERM --> STV = System Table View](#mark_12.0)

   - [Visualización de los estilos de distribución (EVEN, KEY, ALL)](#mark_12.1)

   - [Utilizamos "STV_TBL_PERM" pero para una tabla en especial "encoding_venue"](#mark_12.2)

   - [Continuamos con la sentencia "STV_BLOCKLIST" --> cantidad de bloques de 1 MB](#mark_12.3)

   - [Comparando codificación Raw vs. ZSTD](#mark_12.4)

   - [ANALYZE COMPRESSION, recomienda que tipo de compresión usar.](#mark_12.5)
   
<a name="index_01"></a>

[Estilos de distribución con Redshift](#mark_13)

   - [Distribución AUTO](#mark_13.0)

   - [Distribución EVEN](#mark_13.1)

   - [Distribución KEY](#mark_13.2)

   - [Distribución ALL](#mark_13.3)

[Evaluando los estilos de distribución](#mark_14)

   - [SVV_TABLE_INFO, información resumen de tablas en base de datos](#mark_14.0)
   
   - [SVV_TABLE](#mark_14.0.0)

   - [PG_TABLE_DEF, Almacena información acerca de las columnas de la tabla](#mark_14.1)

   - [SVV_DISKUSAGE, combinación de las tablas STV_TBL_PERM y STV_BLOCKLIST](#mark_14.2)

[sortkey, Llaves de ordenamiento para optimizar nuestras consultas](#mark_15)

   - [Clave de ordenación compuesta (COMPOUND)](#mark_15.0)

   - [Clave de ordenación intercalada (INTERLEAVED)](#mark_15.1)

[Evaluando algoritmos de ordenamiento](#mark_16)

   - [Primer prueba](#test_01)

   - [Segunda prueba](#test_02)

   - [Tercer prueba](#test_03)

   - [Conclusiones](#conclusion_01)

[Buenas prácticas para diseñar tablas en Redshift](#mark_17)

[Tipos de datos en AWS Redshift](#mark_18)

[Reto: mejora el desempeño de tu base de datos](#mark_19)

<a name="index_02"></a>

[Olvídate de los insert, el copy llego para quedarse](#mark_20)

[Cargando archivos tipo JSON](#mark_21)

[El comando copy a fondo](#mark_22)

   - [STL_LOAD_ERRORS](#mark_22.0)
   - [STL_LOAD_COMMITS](#mark_22.1)

[Manifiestos y uso de COMPUPDATE para carga con compresión automática](#mark_23)

   - [Carga con archivo de manifiesto](#mark_23.0)

   - [Comando de compresión COMPUPDATE](#mark_23.1)

[Métodos de carga alternativos al comando copy](#mark_24)

   - [INSERT por lotes](#mark_24.0)

   - [bulk insert, datos por lotes](#mark_24.1)

   - [deep copy](#mark_24.2)

[¿Cómo ejecutar sentencias UPDATE y DELETE?](#mark_25)

[¿Cómo mantener el desempeño de tu base de datos?](#mark_26)

   - [Analyze](#mark_26.0)

   - [Vacuum (Limpieza)](#mark_26.1)

[Estadísticas y limpieza de las tablas](#mark_27)

   - [PG_STATISTIC_INDICATOR](#mark_27.0)

   - [STL_ANALYZE](#mark_27.1)

   - [Vacuum](#mark_27.2)
   
<a name="index_03"></a>

[Agrupamiento, ordenamiento y subqueries](#mark_28)

[¿Qué es y cómo interpretar un explain plan?](#mark_29)

   - [Alerta sobre el performance de las queries con stl_alert_event_log](#mark_29.0)
   
   - [STL_QUERY](#mark_29.1)
   
   - [SVL_QLOG](#mark_29.2)

[¿Cómo descargar datos eficientemente con UNLOAD?](#mark_30)

[Otras tablas útiles de Redshift para entender el comportamiento de nuestros datos](#mark_31)

[](#mark_)

[](#mark_)

[](#mark_)

[](#mark_)

[](#mark_)

[](#mark_)

### <a name="mark_01"></a>¿Que es un Data Warehouse?
    
[Index](#index)

![](img_01.png)

- Es una " gran base de datos" que recibe información de muchas fuentes de datos, su objetivo está definido por una estructura analítica, no mantiene operaciones constantes de por ejemplo I/O sino que está orientado a la analítica de datos.
    
Tomamos los objetos de diferentes fuentes y en diferentes formatos (csv, json, xml, etc) y los llevamos a un solo repositorio "Redshift"
    
![](img_02.png)

Se usa el proceso de ETL (extract, transform, load) para alimentar el Data Warehouse.
    
![](img_03.png)

Extraer: Obtener los datos de las distintas bases de datos
Transformar: Realizar una limpieza y modificación de los datos, creando una buena estructura analítica.
Cargar: Luego de transformar los datos, se cargar al DW
    
### ¿Que es una estructura analítica? 
    
Depende de la organización y sus objetivos, pero el estandar es el modelo dimensional, que posee:
    
![](img_04.png)
    
Modelo estrella, tengo el QUE? en el centro y los CÓMO? alrededor.
    
![](img_05.png)

- Tabla de hechos: Que quiero medir
    
- Tabla de dimensiones: Como medirlo, es decir, que variables son importante para generar los análisis
    
En el curso trabajaremos con el siguiente modelo de AWS Redshift.
    
![](img_06.png)


### <a name="mark_02"></a>Bases de datos columnares y arquitectura orientada a optimización de consultas
    
[Index](#index)

![](img_07.png)
    
### ¿Qué es una base de datos columnar?
    
Redshift es una de ellas. Es una BD optimizada para lograr una recuperación rápida de columnas de datos, normalmente en aplicaciones y con un fin analítico, esto permite procesar queries complejos con cantidades enormes de datos de una manera óptima

### Base de datos basadas en filas (Postgres, MySQL, Oracle)
    
Tienen una arquitectura y un proposito distinto a las bases de datos columnares, están enfocadas en la transaccionalidad y en la lectura y escritura rápida de filas únicas en una db.

![](img_08.png)

Hay una diferencia a nivel de arquitectura en ambas, "EL BLOQUE DE DATOS".
    
![](img_09.png)
    
Cada una de las filas va un bloque de datos guardado en el disco duro, soportando operaciones del tipo transaccional OLTP.
    
![](img_10.png)
    
### Qué pasa si quiero hacer analítica con estas DB:
    
Los bloques de datos de estas db usualmente pesan por defecto 32kb, con lo cual si nuestra data pesa menos, de igual forma el bloque pesará 32kb, en el caso de que nuestra data pese más, ocupará otro bloque de 32kb.
    
Otro punto, si quiero hacer analítica de una tabla de 10 columnas pero solo quiero la información de 2 columnas, de forma inevitable voy a tener 8 columnas que no voy a utilizar pero que sí tengo que consultar por bloque de datos.
    
    
### Base de datos columnar:
    
![](img_11.png)
    
Los registros se guardan columna a columna en un Bloque de Datos, como vemos en el ejemplo anterior la columna SSN serial Number completa se encuentra en un bloque de datos.
    
![](img_12.png)

![](img_13.png)

![](img_14.png)
    
### Nota:
    
Hay algunas bd columnares que son accesibles a travéz de un solo cliente, con lo cual debes aprender a manejar ese cliente en especifico, en Redshift no sucede eso, si manejas DBeaver que es una aplicación de software cliente de SQL y una herramienta de administración de bases de datos, o SQL Workbench, simplemente puedes conectar por medio de HOST, USER, y PASSWORD a la db en Redshift.

### Lecturas recomendadas
    
https://aws.amazon.com/es/nosql/columnar/
    
https://aws.amazon.com/es/redshift/features/

### <a name="mark_03"></a>¿Cómo funciona AWS Redshift?
    
### [Index](#index)

![](img_15.png)

### La clave de Redshift --> Repartir datos:

- La clave de Redshift es "repartir los datos", no voy a instalar Redshift en un Servidor, sino en un cluster siendo este cluster un arreglo de varios servidores conectados y a estos servidores lo voy a llamar Nodos (de datos), cuando me conecte a la db a travéz de los drivers JDBC o ODBC me conectaré a un nodo lider quien orquestará los procesos (repartirá el trabajo en los otros nodos y lo hará en paralelo), luego que los otros nodos hayan computado las tareas el organiza la data y la devuelve al cliente.

### Los Nodos o Servidores:
- Cada nodo es un servidor teniendo su propia memoría RAM, su propio espacio en disco, su propio procesador y está diseñado para procesar muchos datos. Cada nodo tiene unos "Servidores Virtuales, Segmentos, o Slices", entonces cada slice tendrá repartida la data que se le asignó al nodo, un nodo puede tener 2, 4, u 8 slices esto es para que también sea trabajada en paralelo. Hadoop también funciona por cluster y con trabajo en paralelo sobre un File System que para el caso de Hadoop es el HDFS.

### Postgres y Redshift, limitaciones:

- La arquitectura de Postgres 8.02 funciona de base para la creación de Redshift, pero hay algunas cosas que puedo hacer en Postgres que no puedo hacer en Redshift.

Por ejemplo en Redshif no puedo usar:

    - Triggers.
    - Espacios almacenados.
    - Table Spaces.
    - Indices

Pero si analizamos su necesidad para el caso de Redshift no sería muy útiles debido a que estamos pensando en Analítica y esas características son necesarias para cuestiones Transaccionales.

En la siguiete imagen veremos que para una tabla en especifico (tbl que es el id de la tabla), de una columna especifica (col 9) voy a tener la data dividida en 2 slices (0 y 1, estoy trabajando 1 solo nodo), mi bloque de datos en el slice 0 tiene para esa tabla/esa columna tiene 130.994 registros, al igual que el slice 1, podemos ver como se reparte la data en el bloque 0 (blocknum), también podemos ver cual es el valor mín y máx en el slice.

En la imagen de Data Distribuida, podemos ver que para las distintas tablas los slice no manejan exactamente la misma cantidad de filas, lo ideal es repartir la data lo más equitativamente posible para lograr un mejor trabajo en paralelo.

![](img_16.png)


### <a name="mark_04"></a> Creando nuestro entorno de trabajo en AWS

### [Index](#index)

### Creando el Role para Redshift:

Como buena práctica iniciamos creando/limitando un rol para el uso de Redshift.

En la consola de AWS --> IAM

![](img_17.png)

En las opciones por default buscamos "Redshift"

![](img_18.png)

Para estos permisos necesito que Redshift se pueda comunicar con S3.

![](img_19.png)

Colocamos etiquetas si las necesitamos.

![](img_20.png)

![](img_21.png)


### Creando el bucket S3:

Consola de AWS --> S3 --> Crear bucket.

![](img_22.png)

### Configura el cluster de Redshift:

Consola de AWS --> Amazon Redshift

![](img_23.png)

![](img_24.png)

![](img_25.png)

![](img_26.png)

Nombre de la db y un puerto, ambos opcionales.

Master user name --> un nombre de conexión a mi instancia de Redshift y un password
![](img_27.png)

Agregamos el rol creado previamente y luego click en Crear Cluster.

![](img_28.png)


### <a name="mark_05"></a> Cómo conectarnos a Redshift

### [Index](#index)

Primero necesitamos realizar algunas configuraciones para permiter la conexión.

Ingresamos a nuestro cluster.

![](img_29.png)

![](img_30.png)

En la parte final vemos si tenemos acceso público, luego ingresamos al "VPC Security Group".

![](img_31.png)

![](img_32.png)

![](img_33.png)

Agregamos la nueva regla y guardamos los cambios.

![](img_34.png)

Nuevamente en la consola de Redshift, buscamos "View all connection details", donde podremos encontrar los drivers JDBC, ODBC, y la posibilidad de crear un tunel SSH. Elegimos ODBC para este ejemplo.

![](img_35.png)

Para este ejemplo utilizaremos el Cliente DBeaver, es open source y gratuito.

![](img_36.png)

![](img_37.png)

Recordar que para el host --> utilizamos "Server", User --> UID, PWD --> Password

![](img_38.png)

![](img_39.png)

![](img_40.png)

Una vez que testeamos la conexión damos click en Finalizar.

Abrimos un editor para nuestra nueva conexión.

![](img_41.png)

### <a name="mark_06"></a>Creación de tablas en Redshift

### [Index](#index)

En el repo de GitHub tenemos "create_tables_tickit.sql" con el detalle que se puede ver a continuación. En esta creación de tablas existen nuevos "Constrains" para bd columnares `distkey sortkey`.

    - `distkey` es una llave de distribución, distribuye los datos en el custer, en los nodos a travez de los datos que se contienen en la columna o atributo.
    
    -`sortkey` es la llave de ordenamiento facilita la consulta a travéz de los datos que se contienen en la columna o atributo.

Procedemos a crear nuestras tablas en Redshift.

```sql
create table users(
	userid integer not null distkey sortkey,
	username char(8),
	firstname varchar(30),
	lastname varchar(30),
	city varchar(30),
	state char(2),
	email varchar(100),
	phone char(14),
	likesports boolean,
	liketheatre boolean,
	likeconcerts boolean,
	likejazz boolean,
	likeclassical boolean,
	likeopera boolean,
	likerock boolean,
	likevegas boolean,
	likebroadway boolean,
	likemusicals boolean);

create table venue(
	venueid smallint not null distkey sortkey,
	venuename varchar(100),
	venuecity varchar(30),
	venuestate char(2),
	venueseats integer);

create table category(
	catid smallint not null distkey sortkey,
	catgroup varchar(10),
	catname varchar(10),
	catdesc varchar(50));

create table date(
	dateid smallint not null distkey sortkey,
	caldate date not null,
	day character(3) not null,
	week smallint not null,
	month character(5) not null,
	qtr character(5) not null,
	year smallint not null,
	holiday boolean default('N'));

create table event(
	eventid integer not null distkey,
	venueid smallint not null,
	catid smallint not null,
	dateid smallint not null sortkey,
	eventname varchar(200),
	starttime timestamp);

create table listing(
	listid integer not null distkey,
	sellerid integer not null,
	eventid integer not null,
	dateid smallint not null  sortkey,
	numtickets smallint not null,
	priceperticket decimal(8,2),
	totalprice decimal(8,2),
	listtime timestamp);

create table sales(
	salesid integer not null,
	listid integer not null distkey,
	sellerid integer not null,
	buyerid integer not null,
	eventid integer not null,
	dateid smallint not null sortkey,
	qtysold smallint not null,
	pricepaid decimal(8,2),
	commission decimal(8,2),
	saletime timestamp);
```

Las tablas están vacías, necesitamos realizar la ingesta en Redshift.


### Lecturas recomendadas

https://docs.aws.amazon.com/es_es/redshift/latest/dg/c_sampledb.html

https://github.com/alarcon7a/redshift_course/tree/master/Tickit_db

### <a name="mark_07"></a>Carga de datos en S3

### [Index](#index)

Para esto tendremos que cargar los archivos planos tickitdb.zip (descomprimirlos) en un bucket S3.

![](img_42.png)

Dentro de nuestro S3 cargamos o arrastramos y soltamos (drag and drop), una vez realizado ya tenemos los archivos listos para la ingesta.

![](img_43.png)


### Lecturas recomendadas

https://docs.aws.amazon.com/es_es/redshift/latest/dg/c_sampledb.html

https://github.com/alarcon7a/redshift_course/tree/master/Tickit_db

### <a name="mark_08"></a> Ingesta S3 --> Redshift

### [Index](#index)

Desde el archivo de github ejecutamos "copy_s3_to_redshift_tickit.sql" en nuestro cliente (que ya está conectado a Redshift), para realizar la ingesta.

### Importante:

`copy`, lleva data desde un repositorio a las tablas.

Recordar reemplazar `<Reemplazar_con_la_ruta_del_bucket>` con la ruta de tu bucket.

Recordar reemplazar `<Reemplazar_con_tu_iam_role_arn` con el ARN del rol (buscar en IAM el rol creado anteriormente), Redshift tiene que tener el mismo rol.

Recordar reemplazar `<Reemplazar_con_tu_aws_region>` con tu region en AWS.

Archivos **"copy_s3_to_redshift_tickit.sql"** :

```sql
copy users from 's3://<Reemplazar_con_la_ruta_del_bucket>/tickit/allusers_pipe.txt' 
credentials 'aws_iam_role=<Reemplazar_con_tu_iam_role_arn>' 
delimiter '|' region '<Reemplazar_con_tu_aws_region>';

copy venue from 's3://<Reemplazar_con_la_ruta_del_bucket>/tickit/venue_pipe.txt' 
credentials 'aws_iam_role=<Reemplazar_con_tu_iam_role_arn>' 
delimiter '|' region '<Reemplazar_con_tu_aws_region>';

copy category from 's3://<Reemplazar_con_la_ruta_del_bucket>/tickit/category_pipe.txt' 
credentials 'aws_iam_role=<Reemplazar_con_tu_iam_role_arn>' 
delimiter '|' region '<Reemplazar_con_tu_aws_region>';

copy date from 's3://<Reemplazar_con_la_ruta_del_bucket>/tickit/date2008_pipe.txt' 
credentials 'aws_iam_role=<Reemplazar_con_tu_iam_role_arn>' 
delimiter '|' region '<Reemplazar_con_tu_aws_region>';

copy event from 's3://<Reemplazar_con_la_ruta_del_bucket>/tickit/allevents_pipe.txt' 
credentials 'aws_iam_role=<Reemplazar_con_tu_iam_role_arn>' 
delimiter '|' timeformat 'YYYY-MM-DD HH:MI:SS' region '<Reemplazar_con_tu_aws_region>';

copy listing from 's3://<Reemplazar_con_la_ruta_del_bucket>/tickit/listings_pipe.txt' 
credentials 'aws_iam_role=<Reemplazar_con_tu_iam_role_arn>' 
delimiter '|' region '<Reemplazar_con_tu_aws_region>';

copy sales from 's3://<Reemplazar_con_la_ruta_del_bucket>/tickit/sales_tab.txt'
credentials 'aws_iam_role=<Reemplazar_con_tu_iam_role_arn>'
delimiter '\t' timeformat 'MM/DD/YYYY HH:MI:SS' region '<Reemplazar_con_tu_aws_region>';
```
Ejecutamos y las tablas comenzaran a poblar/populate.

### Lecturas recomendadas¶

https://docs.aws.amazon.com/es_es/redshift/latest/dg/c_sampledb.html

https://github.com/alarcon7a/redshift_course/tree/master/Tickit_db

### <a name="mark_09"></a> Algoritmos de compresión con Redshift 1° Parte.

### [Index](#index)

![](img_45.png)

Supongamos que tenemos una tabla que utiliza 500 bloques de datos = 500Mb, si logro comprimirla a 100 bloques de datos, esto se traducirá en mayor velocidad/mejor performance al momento de realizar las lecturas en disco que son transacciones I/O.

Entonces al momento de crear una tabla, no después, debo ejecutar las siguientes sentencias, que basicamente dice que la columna "name" tendrá un ENCODE del tipo TEXT255 (tipo de compresión).

![](img_46.png)

## Algoritmos y tipos de compresión:

### <a name="mark_09.0"></a>RAW - Sin compresión:
[Index](#index)

![](img_47.png)

Raw --> Ninguna compresión, es la data cruda como viene.

### <a name="mark_09.1"></a>AZ64:
[Index](#index)

![](img_48.png)

Como trabaja SIMD.

![](img_49.png)

### <a name="mark_09.2"></a>Codificación por diccionarios de bytes
[Index](#index)

![](img_50.png)

En la codificación por diccionarios de bytes, se crea un diccionario independiente de valores únicos para cada bloque de los valores de columna del disco. (Un bloque de disco de Amazon Redshift ocupa 1 MB). El diccionario tiene hasta 256 valores de un byte que se almacenan como índices de los valores de datos originales. Si se almacenan más de 256 valores en un mismo bloque, los valores adicionales se graban en un bloque descomprimido sin formato. El proceso se repite para cada bloque del disco.

Esta codificación es muy eficaz en columnas de cadenas de cardinalidad baja. Esta codificación es una solución óptima cuando el dominio de los datos de una columna es menor que 256 valores únicos.

En las columnas cuyo tipo de datos de cadena (CHAR y VARCHAR) está codificado con BYTEDICT, Amazon Redshift realiza exámenes vectorizados y evaluaciones de predicados que actúan directamente en los datos comprimidos. Estos exámenes utilizan instrucciones de instrucción única y datos múltiples (SIMD) específicas del hardware para el procesamiento paralelo. Esto acelera considerablemente el examen de las columnas de cadenas. La codificación por diccionario de bytes es muy eficaz en cuanto a espacio si una columna CHAR/VARCHAR almacena cadenas largas de caracteres.

Supongamos que una tabla tiene una columna COUNTRY con un tipo de datos CHAR(30). A medida que se cargan datos, Amazon Redshift crea un diccionario y rellena la columna COUNTRY con el valor índice. El diccionario tiene los valores únicos indexados y la tabla en sí tiene solo los subscripts de un byte de los valores correspondientes.

**nota**

Los espacios a la derecha se almacenan en columnas de caracteres de longitud fija. Por lo tanto, en una columna CHAR (30), cada valor comprimido ahorra 29 bytes de almacenamiento cuando utiliza la codificación por diccionario de bytes.

En la siguiente tabla, se representa el diccionario para la columna COUNTRY.

![](img_53.png)

En la siguiente tabla, se representan los valores de la columna COUNTRY.

|Valor original del dato|	Tamaño original (longitud fija, 30 bytes por valor)	|Valor comprimido (índice)	|Tamaño nuevo (bytes)|
|---|---|---|---|
|England|30|0|1|
|England|30|0|1|
|United States of America|30|1|1|
|United States of America|30|1|1|
|Venezuela|30|2|1|
|Sri Lanka|30|3|1|
|Argentina|30|4|1|
|Japan|30|5|1|
|Sri Lanka|30|3|1|
|Argentina|30|4|1|
|Total|300||10|

El tamaño total comprimido en este ejemplo se calcula de la siguiente manera: hay 6 entradas diferentes almacenadas en el diccionario (6 * 30 = 180) y la tabla tiene 10 valores comprimidos de 1 byte para un total de 190 bytes.

`*************************************************************************************************************************`

En el contexto de Redshift, la codificación por diccionarios de bytes es una técnica de compresión de datos que se utiliza para reducir el tamaño de las columnas de datos que contienen un número limitado de valores únicos.

La codificación por diccionarios de bytes funciona creando un diccionario de los valores únicos que se encuentran en la columna. Luego, cada valor de la columna se reemplaza por su índice en el diccionario. Por ejemplo, si la columna `color` contiene los valores "rojo", "verde", "azul" y "amarillo", el diccionario se vería así:

```
{
  "rojo": 0,
  "verde": 1,
  "azul": 2,
  "amarillo": 3
}
```

Luego, los valores de la columna se pueden almacenar como índices. Por ejemplo, el valor "rojo" se almacenaría como 0, el valor "verde" se almacenaría como 1, etc.

La codificación por diccionarios de bytes puede proporcionar una reducción de tamaño significativa para columnas con un número limitado de valores únicos. Por ejemplo, si la columna `color` contiene 1000 registros, con un promedio de 2 valores únicos por registro, la codificación por diccionarios de bytes puede reducir el tamaño de la columna de 2000 bytes a 1000 bytes.

Para utilizar la codificación por diccionarios de bytes en Redshift, debe habilitar la opción `data_compression` para la columna. Esto se puede hacer mediante la sentencia `ALTER TABLE`. Por ejemplo, para habilitar la codificación por diccionarios de bytes para la columna `color`, se puede utilizar el siguiente comando:

```sql
ALTER TABLE my_table ALTER COLUMN color SET data_compression = 'dict';
```

Una vez que la opción `data_compression` está habilitada, Redshift automáticamente comprimirá la columna utilizando la codificación por diccionarios de bytes.

La codificación por diccionarios de bytes tiene algunas limitaciones. No se puede utilizar para columnas que contienen valores NULL o valores que cambian con frecuencia. Además, la codificación por diccionarios de bytes puede reducir la velocidad de las consultas que utilizan la columna.

En general, la codificación por diccionarios de bytes es una técnica eficaz para reducir el tamaño de las columnas de datos que contienen un número limitado de valores únicos. Sin embargo, es importante tener en cuenta las limitaciones de la técnica antes de utilizarla.

`*************************************************************************************************************************`

### <a name="mark_09.3"></a>Codificacion Delta:
[Index](#index)

![](img_51.png)

Se basa en medir la distancia entre un dato y el siguiente.

![](img_52.png)

Si el delta supera número de dígitos posibles que puedo guardar en 1 byte, que es desde -127 hasta 127 no se podrá codificar, de igual forma para el de 2 bytes, será desde -32.000 hasta 32.000.

`*************************************************************************************************************************`

La codificación delta es una técnica de compresión de datos que se utiliza para almacenar las diferencias entre dos versiones de un archivo. Esto permite almacenar solo los cambios que se han realizado en el archivo, en lugar de almacenar todo el archivo nuevamente.

La codificación delta funciona comparando las dos versiones del archivo y generando un archivo que contiene las diferencias entre ellas. Estas diferencias se pueden almacenar de varias maneras, como:

* **Diferencias de bytes:** Esta es la forma más simple de codificación delta. Simplemente se almacenan los bytes que han cambiado entre las dos versiones del archivo.
* **Diferencias de registros:** Esta forma de codificación delta almacena las diferencias entre los registros de las dos versiones del archivo.
* **Diferencias de estructuras de datos:** Esta forma de codificación delta almacena las diferencias entre las estructuras de datos de las dos versiones del archivo.

La codificación delta puede proporcionar una reducción de tamaño significativa para archivos que cambian con frecuencia. Por ejemplo, si un archivo se modifica agregando un nuevo registro al final, la codificación delta puede reducir el tamaño del archivo en un 99%.

La codificación delta se utiliza en una variedad de aplicaciones, como:

* **Versionado de archivos:** La codificación delta se puede utilizar para almacenar las versiones anteriores de un archivo. Esto permite restaurar una versión anterior del archivo si es necesario.
* **Copia de seguridad incremental:** La codificación delta se puede utilizar para realizar copias de seguridad incrementales de un archivo. Esto reduce el tiempo y el espacio de almacenamiento necesarios para realizar copias de seguridad.
* **Carga de datos:** La codificación delta se puede utilizar para cargar datos de forma incremental en una base de datos. Esto reduce el tiempo necesario para cargar los datos.

En el contexto de Amazon Redshift, la codificación delta se utiliza para almacenar las diferencias entre dos versiones de un conjunto de datos. Esto permite a Redshift cargar los datos incrementalmente, lo que reduce el tiempo necesario para cargar los datos.

Para utilizar la codificación delta en Redshift, debe habilitar la opción `data_compression` para el conjunto de datos. Esto se puede hacer mediante la sentencia `ALTER TABLE`. Por ejemplo, para habilitar la codificación delta para el conjunto de datos `my_data`, se puede utilizar el siguiente comando:

```sql
ALTER TABLE my_data ALTER COLUMN data SET data_compression = 'delta';
```

Una vez que la opción `data_compression` está habilitada, Redshift automáticamente comprimirá el conjunto de datos utilizando la codificación delta.

`*************************************************************************************************************************`

### <a name="mark_09.4"></a>Codificación LZO
[Index](#index)

![](img_54.png)

Cadenas de texto libre --> descripciones.


### Lecturas recomendadas

https://docs.aws.amazon.com/redshift/latest/dg/c_Compression_encodings.html

https://aws.amazon.com/es/about-aws/whats-new/2019/10/amazon-redshift-introduces-az64-a-new-compression-encoding-for-optimized-storage-and-high-query-performance/

https://docs.aws.amazon.com/es_es/redshift/latest/dg/c_Byte_dictionary_encoding.html

### <a name="mark_10"></a> Algoritmos de compresión con Redshift 2° Parte.

### [Index](#index)


### <a name="mark_10.0"></a>Codificación Mostly.
[Index](#index)

Las codificaciones mostly son útiles cuando el tipo de datos de una columna es mayor que lo que requieren la mayoría de los valores almacenados. Al especificar una codificación mostly para este tipo de columnas, puede comprimir la mayoría de los valores de la columna a un tamaño de almacenamiento estándar más pequeño. Los valores restantes que no pueden comprimirse se almacenan sin formato. Por ejemplo, puede comprimir una columna de 16 bits, como una columna IT2, a un almacenamiento de 8 bits.

Por lo general, las codificaciones mostly funcionan con los siguientes tipos de datos:

SMALLINT/INT2 (16 bits)

INTEGER/INT (32 bits)

BIGINT/INT8 (64 bits)

DECIMAL/NUMERIC (64 bits)

Seleccione la variación adecuada de codificación mostly en función del tamaño de los tipos de datos de la columna. Por ejemplo, aplique la codificación MOSTLY8 a una columna definida para valores enteros de 16 bits. No está permitido aplicar MOSTLY16 a una columna con tipos de datos de 16 bits o MOSTLY32 a una columna con tipos de datos de 32 bits.

La mayoría de las codificaciones pueden ser menos eficaces que la ausencia de compresión cuando no se puede comprimir un número relativamente elevado de los valores de la columna. Antes de aplicar una de estas codificaciones a una columna, realice la comprobación correspondiente. La mayoría de los valores que se vayan a cargar ahora (y que es probable que se carguen más adelante) deberían ajustarse a los rangos que se muestran en la siguiente tabla.

![](img_55.png)

#### nota

`Para los valores decimales, omita la coma decimal para determinar si el valor se ajusta al rango. Por ejemplo, 1 234,56 es tratado como 123 456 y se puede comprimir en una columna MOSTLY32.`

Por ejemplo, la columna VENUEID de la tabla VENUE está definida como una columna de valores enteros sin formato, lo que significa que su valor consume 4 bytes de almacenamiento. No obstante, el rango actual de valores de la columna es de 0 a 309. Por lo tanto, si se vuelve a crear y a cargar esta tabla con la codificación MOSTLY16 para VENUEID, se reduciría el almacenamiento de cada valor en esa columna a 2 bytes.

Si la mayoría de los valores de VENUEID a los que se hace referencia en otra tabla estuvieran en el rango de 0 a 127, podría tener sentido codificar esa columna de clave externa como MOSTLY8. Antes de tomar una decisión, ejecute algunas consultas en los datos de la tabla de referencia para averiguar si la mayoría de los valores están comprendidos en el intervalo de 8 bits, 16 bits o 32 bits.

En la siguiente tabla, se muestran los tamaños comprimidos para valores numéricos específicos cuando se utilizan codificaciones MOSTLY8, MOSTLY16 y MOSTLY32:

![](img_56.png)

### <a name="mark_10.1"></a>Codificación de run length
[Index](#index)

La codificación de run length reemplaza un valor que se repite de manera consecutiva por un token que consiste en el valor y un recuento de la cantidad de ocurrencias consecutivas (la longitud de la ejecución). Se crea un diccionario independiente de valores únicos para cada bloque de los valores de columna del disco. (Un bloque de disco de Amazon Redshift ocupa 1 MB). Esta codificación es una opción ideal para una tabla en la que los valores de datos suelen repetirse de manera consecutiva; por ejemplo, cuando la tabla está ordenada según esos valores.

Por ejemplo, imagine que una columna de una tabla de grandes dimensiones presenta un dominio previsiblemente pequeño, como una columna COLOR con menos de 10 valores posibles. Es probable que estos valores queden dispuestos en secuencias de gran longitud en toda la tabla, incluso si los datos no están ordenados.

No recomendamos aplicar la codificación de run length en ninguna columna que esté designada como clave de ordenación. Los exámenes de rango restringido funcionan mejor cuando los bloques tienen una cantidad similar de filas. Si las columnas de clave de ordenación se comprimen mucho más que otras columnas en la misma consulta, los exámenes de rango restringido podrían tener un rendimiento deficiente.

En la siguiente tabla se utiliza el ejemplo de la columna COLOR para mostrar cómo funciona la codificación de run length.

![](img_57.png)

### <a name="mark_10.2"></a>Codificaciones Text255 y Text32k
[Index](#index)

Las codificaciones text255 y text32k son útiles para comprimir columnas VARCHAR en las que se repiten con frecuencia las mismas palabras. Se crea un diccionario independiente de palabras únicas para cada bloque de los valores de columna del disco. (Un bloque de disco de Amazon Redshift ocupa 1 MB). El diccionario tiene las primeras 245 palabras únicas de la columna. Estas palabras se reemplazan en el disco por un valor índice de un byte, lo que representa uno de 245 valores, y todas las palabras que no están representadas en el diccionario se almacenan sin comprimir. El proceso se repite para cada bloque de 1 MB del disco. Si las palabras indexadas se repiten con frecuencia en la columna, esta arrojará una alta relación de compresión.

Para la codificación text32k, se aplica el mismo principio, pero el diccionario para cada bloque no captura una cantidad específica de palabras. En lugar de ello, el diccionario indexa cada palabra única que encuentra hasta que las entradas combinadas alcancen una longitud de 32K, menos alguna sobrecarga. Los valores índices se almacenan en dos bytes.

Por ejemplo, veamos el caso de la columna VENUENAME en la tabla VENUE. Palabras como Arena, Center y Theatre son recurrentes en esta columna y es posible que estén dentro de las primeras 245 palabras que se encuentran en cada bloque si se aplica la compresión text255. En tal caso, esta columna se beneficia de la compresión. El motivo de ello es que cada vez que aparezcan esas palabras, ocuparán solo 1 byte de almacenamiento (en lugar de 5, 6 o 7 bytes, respectivamente).

### <a name="mark_10.3"></a>Codificación ZSTD
[Index](#index)

La codificación zstandard (ZSTD) proporciona una relación de compresión muy alta con un muy buen rendimiento en diferentes conjuntos de datos. La codificación ZSTD funciona especialmente bien para las columnas CHAR y VARCHAR que almacenan una gran variedad de cadenas de caracteres largas y cortas, como descripciones de productos, comentarios de usuarios, registros o cadenas JSON. Mientras que algunos algoritmos, como la codificación Delta o la codificación Mostly pueden, potencialmente, usar más espacios de almacenamiento que si se dejan los datos sin comprimir, es muy poco probable que ZSTD aumente el uso del disco.

ZSTD admite los tipos de datos SMALLINT, INTEGER, BIGINT, DECIMAL, REAL, DOUBLE PRECISION, BOOLEAN, CHAR, VARCHAR, DATE, TIMESTAMP y TIMESTAMPTZ.

### Conclusión:

![](img_58.png)

### Lecturas recomendadas

https://docs.aws.amazon.com/redshift/latest/dg/c_Compression_encodings.html

https://aws.amazon.com/es/about-aws/whats-new/2019/10/amazon-redshift-introduces-az64-a-new-compression-encoding-for-optimized-storage-and-high-query-performance/

### <a name="mark_11"></a> Codificado/Compresión de Columnas.

### [Index](#index)

En el siguiente ejemplo tenemos una tabla con 38 millones de registros, en la columna "venuename" tendremos nombres de personas, la prueba realizada es crear una nueva tabla "encoding_venue" con 7 diferentes tipos de codificación para los datos de la columna "venuename".

```sql
CREATE table encoding_venue(
nameraw varchar(100) encode raw,
namebytedict varchar(100) encode bytedict,
namelzo varchar(100) encode lzo,
namerunlength varchar(100) encode runlength,
nametext255 varchar(100) encode text255,
nametext32k varchar(100) encode text32k,
namezstd varchar(100) encode zstd,

);
```

Populando la tabla "encoding_venue" con todos los valores de los nombres, el siguiente ejemplo inserta dentro de "encoding_venue", a travéz de un SELECT 7 columnas de valores "venuename" que vienen desde la tabla "cartesian_venue", de esta forma inserta 38 millones de filas repetidas en 7 columnas pero cada columna con su propio tipo de codificación.

```sql

INSERT INTO encoding_venue
(
SELECT venuename,venuename,venuename,venuename,venuename,venuename,venuename
FROM cartesian_venue
);

```

### Observación:

Una codificación de compresión especifica el tipo de compresión que se aplica a una columna de valores de datos a medida que se añaden filas a una tabla.

Si no se especifica una compresión en la instrucción CREATE TABLE o ALTER TABLE, Amazon Redshift asigna automáticamente la codificación de compresión de la siguiente manera:

A las columnas que están definidas como claves de ordenación se les asigna una compresión RAW.

A las columnas que están definidas como tipos de datos BOOLEAN, REAL o DOUBLE PRECISION se les asigna una compresión RAW.

Las columnas que se definen como tipos de datos SMALLINT, INTEGER, BIGINT, DECIMAL, CHAR, VARCHAR, DATE, TIMESTAMP o TIMESTAMPTZ tienen asignada la compresión AZ64.

Las columnas que se definen como tipos de datos CHAR o VARCHAR tienen asignada la compresión LZO.

### <a name="mark_12"></a> Análisis de desempeño con diferentes tipos de compresión

### [Index](#index)

![](img_59.png)

### <a name="mark_12.0"></a>STV_TBL_PERM --> STV = System Table View
### [Index](#index)

La tabla STV_TBL_PERM tiene información acerca de las tablas permanentes en Amazon Redshift, incluidas las tablas temporales creadas por un usuario para esta sesión. STV_TBL_PERM tiene información de todas las tablas en todas las bases de datos.

Esta tabla se diferencia de STV_TBL_TRANS, la cual tiene información relacionada con las tablas transitorias de bases de datos que el sistema crea durante el procesamiento de consultas.

Solo los superusuarios pueden ver STV_TBL_PERM. Para obtener más información, consulte Visibilidad de datos en las tablas y vistas de sistema.

Columnas de la tabla:

|Nombre de la columna | Tipo de datos | Descripción |
|:---|:---:|:---|
|slice|entero|Sector del nodo asignado a la tabla.|
|id|entero|ID de la tabla.|
|nombre|character (72)|	Nombre de la tabla.|
|rows|bigint|Cantidad de fila de datos en el sector/slice.|
|sorted_rows|bigint|Cantidad de filas en el sector que ya están ordenadas en el disco. Si esta cantidad no coincide con la cantidad indicada en ROWS, limpie la tabla para volver a ordenar las filas.|
|temp|entero|Indica si es una tabla temporal o no: 0 = false; 1 = true.|
|db_id|entero|ID de la base de datos donde se crea la tabla.|
|insert_pristine|entero|Para uso interno.|
|delete_pristine|entero|Para uso interno.|
|backup|entero|Valor que indica si la tabla se incluye en las instantáneas del clúster: 0 = no; 1 = sí. Para obtener más información, consulte el parámetro BACKUP del comando CREATE TABLE.|
|dist_style|entero|Estilo de distribución de la tabla a la que pertenece el segmento. Para obtener más información acerca de los valores, consulte Visualización de los estilos de distribución.|
|block_count|entero|Número de bloques que utiliza el segmento. El valor es -1 cuando no se puede calcular el número de bloques.|

### <a name="mark_12.1"></a> Visualización de los estilos de distribución (EVEN, KEY, ALL)
### [Index](#index)

La columna RELEFFECTIVEDISTSTYLE en PG_CLASS_INFO indica el estilo de distribución actual de la tabla. Si la tabla usa una distribución automática, RELEFFECTIVEDISTSTYLE es 10, 11 o 12, lo que indica que el estilo de distribución efectivo es AUTO (ALL) o AUTO (EVEN) o AUTO (KEY). Si la tabla usa distribución automática, el estilo de distribución podría inicialmente mostrar AUTO (ALL), para a continuación mostrar AUTO (EVEN) o AUTO (KEY) cuando la tabla crezca.

En la siguiente tabla, se proporciona el estilo de distribución para cada valor de la tabla RELEFFECTIVEDISTSTYLE:

|RELEFFECTIVEDISTSTYLE|	Estilo de distribución actual|
|:---|:---|
|0|	EVEN|
|1|	KEY|
|8|	ALL|
|10|	AUTO (ALL)|
|11|	AUTO (EVEN)|
|12|	AUTO (KEY)|

La columna DISTSTYLE en SVV_TABLE_INFO indica el estilo de distribución actual de la tabla. Si la tabla usa una distribución automática, DISTSTYLE es AUTO (ALL), AUTO (EVEN) o AUTO (KEY).

En el siguiente ejemplo, se crean cuatro tablas usando los tres estilos de distribución y distribución automática, luego, se consulta la tabla SVV_TABLE_INFO para ver los estilos de distribución.

```sql
create table public.dist_key (col1 int)
diststyle key distkey (col1);

insert into public.dist_key values (1);

create table public.dist_even (col1 int)
diststyle even;

insert into public.dist_even values (1);

create table public.dist_all (col1 int)
diststyle all;

insert into public.dist_all values (1);

create table public.dist_auto (col1 int);

insert into public.dist_auto values (1);

select "schema", "table", diststyle from SVV_TABLE_INFO
where "table" like 'dist%';
```

        schema   |    table        | diststyle
     ------------+-----------------+------------
      public     | dist_key        | KEY(col1)
      public     | dist_even       | EVEN
      public     | dist_all        | ALL
      public     | dist_auto       | AUTO(ALL)
      
      
### <a name="mark_12.2"></a> Utilizamos "STV_TBL_PERM" pero para una tabla en especial "encoding_venue":
### [Index](#index)

Como podemos ver, tabla "encoding_venue" se encuentra distribuida en 4 slices, si tenemos 2 nodos (2 slices por nodo), conteniendo 9,721,099 filas por cada uno de los slices, copio el id de mi tabla (sin la coma) "101395"

![](img_60.png)

### <a name="mark_12.3"></a>Continuamos con la sentencia "STV_BLOCKLIST" --> cantidad de bloques de 1 MB.
### [Index](#index)

![](img_61.png)

Realizamos la misma búsquda pero para el id de la tabla que habíamos copiado "101395".

![](img_62.png)

Recordemos los tipos de codificación que utilizamos al crear la tabla:

```sql
CREATE table encoding_venue(
nameraw varchar(100) encode raw, --0
namebytedict varchar(100) encode bytedict, --1
namelzo varchar(100) encode lzo, --2
namerunlength varchar(100) encode runlength,--3
nametext255 varchar(100) encode text255,--4
nametext32k varchar(100) encode text32k,--5
namezstd varchar(100) encode zstd,--6

);
```

O se que para la codificación "0" (raw: sin coprimir) estoy usando 203 bloques de datos por cada segmento/sector/slice en nuestros nodos, 

La columna "1" y "6" (compresión bytedict, zstd respectivamente) solo ocupa 10 bloques de datos, las mejores, las otras no son tan buenas y "2" (lzo) que incrementó el valor de bloques.

### <a name="mark_12.4"></a>Comparando codificación Raw vs. ZSTD.
### [Index](#index)

Col = 0 --> Raw, tenemos 47,759 valores por cada bloque.

![](img_63.png)

Col = 6 --> ZSTD, tenemos 982,383 valores por cada bloque.

![](img_64.png)


STV_BLOCKLIST tiene la cantidad de bloques de 1 MB de disco que utiliza cada sector, tabla o columna en una base de datos.

Utilice consultas de agregación con STV_BLOCKLIST, como se muestra en los siguientes ejemplos, para determinar la cantidad de bloques de 1 MB de disco asignados para cada base de datos, tabla, sector o columna. También puede utilizar [STV_PARTITIONS](https://docs.aws.amazon.com/es_es/redshift/latest/dg/r_STV_PARTITIONS.html) para obtener información resumida acerca de la utilización del disco.

Solo los superusuarios pueden ver STV_BLOCKLIST. Para obtener más información, consulte [Visibilidad de datos en las tablas y vistas de sistema](https://docs.aws.amazon.com/es_es/redshift/latest/dg/cm_chap_system-tables.html#c_visibility-of-data).

#### Columnas de la tabla:

|Nombre de la columna|	Tipo de datos|	Descripción|
|:---|:---:|:---|
|slice|	entero|	Sector del nodo.|
|col|	entero|	Índice con base cero para la columna. Cada tabla que cree tiene tres columnas ocultas anexadas: INSERT_XID, DELETE_XID y ROW_ID (OID). Una tabla con 3 columnas definidas por el usuario tiene 6 columnas reales, y las columnas definidas por el usuario se enumeran internamente como 0, 1 y 2. Las columnas INSERT_XID, DELETE_XID y ROW_ID se enumeran 3, 4 y 5, respectivamente, en este ejemplo.|
|tbl|	entero|	ID de la tabla para la tabla de la base de datos.|
|blocknum|	entero|	ID para el bloque de datos.|
|num_values|	entero|	Cantidad de valores contenidos en el bloque.|
|extended_limits|	entero|	Para uso interno.|
|minvalue|	bigint|	Valor mínimo de datos del bloque. Almacena los primeros ocho caracteres como un entero de 64 bits para datos no numéricos. Se utiliza para explorar el disco.|
|maxvalue|	bigint	|Valor máximo de dato del bloque. Almacena los primeros ocho caracteres como un entero de 64 bits para datos no numéricos. Se utiliza para explorar el disco.|
|sb_pos|	entero|	Identificador interno de Amazon Redshift para la posición del superbloque en el disco.|
|pinned|	entero|	Indica si el bloque se conectó o no a la memoria como parte de la carga previa: 0 = false; 1 = true. La opción predeterminada es falso.|
|on_disk|	entero|	Indica si el bloque se almacenó automáticamente o no en el disco: 0 = false; 1 = true. La opción predeterminada es falso.|
|modified|	entero|	Indica si el bloque se modificó o no: 0 = false; 1 = true. La opción predeterminada es falso.|
hdr_modified|	entero|	Indica si el encabezado del bloque se modificó o no: 0 = false; 1 = true. La opción predeterminada es falso.|
|unsorted|	entero|	Indica si el bloque está desordenado o no: 0 = false; 1 = true. El valor predeterminado es verdadero.|
|tombstone|	entero|	Para uso interno.|
|preferred_diskno|	entero	|Cantidad de discos en que debe estar el bloque, excepto que el disco tenga una falla. Una vez arreglado el disco, el bloque volverá a ese disco.|
|temporary|	entero	|Indica si el bloque tiene o no datos temporales, como una tabla temporal o resultados intermedios de consulta: 0 = false; 1 = true. La opción predeterminada es falso.|
|newblock|	entero	|Indica si un bloque es o no nuevo (true) o si nunca se guardó en el disco (false): 0 = false; 1 = true.|
|num_readers|	entero	|Cantidad de referencias en cada bloque.|
|flags|	entero	|Indicadores internos de Amazon Redshift para el encabezado del bloque.|

### Consultas de ejemplo:

STV_BLOCKLIST tiene una fila por cada bloque del disco designado, por lo que una consulta que selecciona todas las filas posiblemente devuelva una gran cantidad de filas. Le recomendamos usar solo las consultas de agregación con STV_BLOCKLIST.

La vista [SVV_DISKUSAGE](https://docs.aws.amazon.com/es_es/redshift/latest/dg/r_SVV_DISKUSAGE.html) proporciona información similar en un formato más sencillo de usar. Sin embargo, en el siguiente ejemplo se demuestra un uso de la tabla STV_BLOCKLIST.

Para determinar la cantidad de bloques de 1 MB que utiliza cada columna de la tabla VENUE, escriba la siguiente consulta:

```sql

select col, count(*)
from stv_blocklist, stv_tbl_perm
where stv_blocklist.tbl = stv_tbl_perm.id
and stv_blocklist.slice = stv_tbl_perm.slice
and stv_tbl_perm.name = 'venue'
group by col
order by col;

```

Esta consulta devuelve la cantidad de bloques de 1 MB asignados a cada columna de la tabla VENUE, tal como se muestra en los siguientes datos de ejemplo:

 |col | count|
|:---:|:---:|
  | 0 |  4|
  | 1 |  4|
  | 2 |  4|
  | 3 |  4|
  | 4 |  4|
  | 5 |  4|
  | 7 |  4|
  | 8 |  4|
   
(8 rows)

En la siguiente consulta, se muestra si una tabla está distribuida en todos los sectores:

```sql

select trim(name) as table, stv_blocklist.slice, stv_tbl_perm.rows
from stv_blocklist,stv_tbl_perm
where stv_blocklist.tbl=stv_tbl_perm.id
and stv_tbl_perm.slice=stv_blocklist.slice
and stv_blocklist.id > 10000 and name not like '%#m%'
and name not like 'systable%'
group by name, stv_blocklist.slice, stv_tbl_perm.rows
order by 3 desc;

```

Esta consulta produce el siguiente ejemplo de salida, que muestra una distribución uniforme de los datos en la tabla que tiene la mayor cantidad de filas:

|table   | slice | rows|
|:---|:---:|:---:|
|listing  |    13 | 10527|
|listing  |    14 | 10526|
|listing  |     8 | 10526|
|listing  |     9 | 10526|
|listing  |     7 | 10525|
|listing  |     4 | 10525|
|listing  |    17 | 10525|
|listing  |    11 | 10525|
|listing  |     5 | 10525|
|listing  |    18 | 10525|
|listing  |    12 | 10525|
|listing  |     3 | 10525|
|listing  |    10 | 10525|
|listing  |     2 | 10524|
|listing  |    15 | 10524|
|listing  |    16 | 10524|
|listing  |     6 | 10524|
|listing  |    19 | 10524|
|listing  |     1 | 10523|
|listing  |     0 | 10521|
...
(180 rows)

La siguiente consulta determina si hay algún bloque del tipo tombstone confirmado en el disco:

```sql
select slice, col, tbl, blocknum, newblock
from stv_blocklist
where  tombstone > 0;
```


|slice | col |   tbl  | blocknum | newblock|
|:---:|:---:|:---:|:---:|:---:|
|4     |  0  | 101285 |    0     |   1|
|4     |  2  | 101285 |    0     |   1|
|4     |  4  | 101285 |    1     |   1|
|5     |  2  | 101285 |    0     |   1|
|5     |  0  | 101285 |    0     |   1|
|5     |  1  | 101285 |    0     |   1|
|5     |  4  | 101285 |    1     |   1|
...
(24 rows)

### <a name="mark_12.5"></a>ANALYZE COMPRESSION, recomienda que tipo de compresión usar.
### [Index](#index)

Realiza un análisis de compresión y produce un informe con la codificación de compresión sugerida para las tablas analizadas. Para cada columna, el informe incluye un cálculo de la reducción potencial de espacio en disco en comparación con la codificación actual.

![](img_65.png)




### Lectura recomendada:

https://docs.aws.amazon.com/es_es/redshift/latest/dg/c_intro_STV_tables.html

### <a name="mark_13"></a>Estilos de distribución con Redshift

### [Index](#index_01)

![](img_66.png)

![](img_67.png)

### Las tres tipos de distribuciones disponibles en Redshift:

También podemos ver [estilos de distribuciones](#mark_12.1)

![](img_68.png)

Cuando crea una tabla, puede designar uno de los cuatro estilos de distribución: **AUTO, EVEN, KEY o ALL**.

Si no se especifica un estilo de distribución, Amazon Redshift usa la **distribución AUTO**.

### <a name="mark_13.0"></a>Distribución AUTO 
[Index](#index_01)

Con la distribución AUTO, Amazon Redshift asigna un estilo de distribución óptimo basado en el tamaño de los datos de la tabla. Por ejemplo, si se especifica el estilo de distribución AUTO, Amazon Redshift asigna inicialmente el estilo de distribución ALL a una tabla pequeña. Cuando la tabla crezca, Amazon Redshift podría cambiar el estilo de distribución a KEY y elegir la clave principal (o una columna de la clave primaria compuesta) como la clave de distribución. Si la tabla crece y ninguna de las columnas es adecuada para ser la clave de distribución, Amazon Redshift cambia el estilo de distribución a EVEN. El cambio en el estilo de distribución se produce en segundo plano y tiene un impacto mínimo en las consultas de los usuarios.

Si desea consultar las acciones que Amazon Redshift realizó de forma automática para modificar la clave de distribución de una tabla, consulte [SVL_AUTO_WORKER_ACTION](https://docs.aws.amazon.com/es_es/redshift/latest/dg/r_SVL_AUTO_WORKER_ACTION.html). Si desea conocer las recomendaciones actuales relativas a la modificación de la clave de distribución de una tabla, consulte [SVV_ALTER_TABLE_RECOMMENDATIONS](https://docs.aws.amazon.com/es_es/redshift/latest/dg/r_SVV_ALTER_TABLE_RECOMMENDATIONS.html).

Para ver el estilo de distribución aplicado a una tabla, consulte la vista de catálogo del sistema PG_CLASS_INFO. Para obtener más información, consulte [Visualización de los estilos de distribución](https://docs.aws.amazon.com/es_es/redshift/latest/dg/viewing-distribution-styles.html). Si no se especifica un estilo de distribución con la instrucción CREATE TABLE, Amazon Redshift aplica la distribución AUTO.

### <a name="mark_13.1"></a>Distribución EVEN
[Index](#index_01)

![](img_71.png)

El nodo principal distribuye las filas entre los sectores con un método de turnos rotativos, independientemente de los valores de cualquier columna en particular. La distribución EVEN resulta adecuada cuando una tabla no participa en las uniones. Además, resulta adecuada en los casos en que no hay una opción clara entre la distribución KEY y la distribución ALL.

Algoritmo utilizado: round-robin

`*************************************************************************************************************************`

El algoritmo de planificación Round-Robin (RR) es un algoritmo de planificación de procesos simple y justo, que se utiliza en sistemas operativos multitarea. El algoritmo RR asigna a cada proceso una cantidad de tiempo de CPU denominada **quantum**. Cuando un proceso se ejecuta, se le asigna el control de la CPU durante el quantum. Al final del quantum, el proceso se suspende y el siguiente proceso en la cola de "listos" se ejecuta. El proceso que se suspendió se coloca de nuevo en la cola de "listos" y se ejecutará de nuevo cuando le llegue su turno.

El algoritmo RR es justo porque cada proceso tiene la oportunidad de ejecutarse durante un quantum, independientemente de su prioridad. Sin embargo, el algoritmo RR puede no ser eficiente si los procesos tienen diferentes necesidades de CPU. Por ejemplo, un proceso que necesita más tiempo de CPU que el quantum podría terminar ejecutándose de manera ineficiente.

El algoritmo RR funciona de la siguiente manera:

1. Se crea una cola de listos que contiene todos los procesos listos para ejecutarse.
2. El proceso en la parte superior de la cola de listos se ejecuta durante el quantum.
3. Al final del quantum, el proceso se suspende y el siguiente proceso en la cola de listos se ejecuta.
4. El proceso que se suspendió se coloca de nuevo en la cola de listos.
5. Los pasos 2-4 se repiten hasta que todos los procesos hayan terminado de ejecutarse.

El algoritmo RR se puede implementar utilizando una cola circular. La cola circular es una estructura de datos que se utiliza para almacenar un conjunto de datos en el que el primer elemento y el último elemento están conectados. Esto permite que el algoritmo RR acceda al primer elemento de la cola sin tener que desplazarse por la cola completa.

El algoritmo RR es un algoritmo simple y eficiente que se utiliza en una amplia variedad de sistemas operativos. Es un buen algoritmo para sistemas operativos multitarea en los que todos los procesos tienen las mismas necesidades de CPU.

`*************************************************************************************************************************`

### <a name="mark_13.2"></a>Distribución KEY
[Index](#index_01)

![](img_69.png)

Las filas se distribuyen según los valores de una columna (valores clave). El nodo principal ubica juntos los valores que coinciden en el mismo sector del nodo. Si distribuye un par de tablas en las claves de unión, el nodo principal ubica las filas en los sectores según los valores de las columnas de unión. De este modo, los valores que coinciden de las columnas que tienen en común se almacenan juntos físicamente.

### <a name="mark_13.3"></a>Distribución ALL
[Index](#index_01)

![](img_70.png)

Se distribuye una copia de toda la tabla a cada nodo. Mientras que la distribución EVEN o la distribución KEY colocan solo una parte de las filas de la tabla en cada nodo, la distribución ALL garantiza que se coloque cada fila para cada combinación en la que participa la tabla.

La distribución ALL multiplica el almacenamiento requerido por la cantidad de nodos del clúster, por lo que demanda más tiempo para cargar, actualizar o insertar datos en distintas tablas. La distribución ALL es adecuada solo para tablas con movimientos relativamente lentos, es decir tablas que no se actualizan con frecuencia ni de forma generalizada. Dado que el costo de redistribuir tablas pequeñas durante una consulta es bajo, no hay un beneficio significativo para definir tablas de dimensiones pequeñas como DISTSTYLE ALL.

#### nota:

Una vez que haya especificado un estilo de distribución para una columna, Amazon Redshift se encarga de la distribución de datos en el nivel del clúster. Amazon Redshift no requiere ni admite el concepto de partición de datos dentro de los objetos de la base de datos. No es necesario crear espacios de tablas ni definir esquemas de partición para las tablas.

En ciertas situaciones, puede cambiar el estilo de distribución de una tabla después de crearla. Para obtener más información, consulte [ALTER TABLE](https://docs.aws.amazon.com/es_es/redshift/latest/dg/r_ALTER_TABLE.html). En aquellas situaciones en las que no puede cambiar el estilo de distribución de una tabla después de crearla, puede volver a crear la tabla y rellenar la nueva tabla con una copia profunda. Para obtener más información, consulte [Realización de una copia profunda](https://docs.aws.amazon.com/es_es/redshift/latest/dg/performing-a-deep-copy.html).


### Lectura Recomendada:

https://docs.aws.amazon.com/es_es/redshift/latest/dg/c_choosing_dist_sort.html

### <a name="mark_14"></a>Evaluando los estilos de distribución

### [Index](#index_01)

## Importante!

**Recordar que los estilos de distribución y la compresión son establecidos al momento de la creación de la tabla.**

Como ejemplo crearemos un tablas simples con 1 sola columna y cada uno de los estilos de distribución

```sql
CREATE TABLE dist_key (columna int)
diststyle key distkey (columna); -- Necesito definir la columna clave
INSERT INTO dist_key VALUES (10);

CREATE TABLE dist_even (columna int)
diststyle even; -- No necesito definir la columna clave, ya que usa el algorito de Round-Robin.
INSERT INTO dist_even VALUES (10);

CREATE TABLE dist_all (columna int)
diststyle all; -- No necesito definir la columna clave, ya que distribuye todas las tablas en cada nodo.
INSERT INTO dist_all VALUES (10);

CREATE TABLE dist_auto (columna int);-- Evaluando la distribución automática
INSERT INTO dist_auto VALUES (10)
```

### <a name="mark_14.0"></a>SVV_TABLE_INFO, información resumen de tablas en base de datos.
[Index](#index_01)

Muestra información de resumen acerca de las tablas en la base de datos. La vista filtra las tablas de sistema y muestra solamente las tablas definidas por el usuario.

Columnas de la tabla

|Nombre de la columna|	Tipo de datos|	Descripción|
|:---|:---:|:---|
|database	|texto	|Nombre de la base de datos.|
|schema	|texto	|Nombre del esquema.|
|table_id	|oid	|ID de la tabla.|
|table|	texto	|Nombre de la tabla.|
|encoded	|texto	|Valor que indica si la columna tiene definida alguna codificación de compresión.|
|diststyle	|texto	|Estilo de distribución o columna de clave de distribución, siempre que se haya definido una clave de distribución. Entre los valores posibles se incluyen EVEN, KEY(column), ALL, AUTO(ALL), AUTO(EVEN) y AUTO(KEY(column)).|
|sortkey1	|texto	|Primera columna en la clave de ordenación, siempre que se haya definido una clave de ordenación. Entre los valores posibles se incluyen: column, AUTO(SORTKEY) y AUTO(SORTKEY(column)).|
|max_varchar	|entero	|Tamaño de la columna más grande que utiliza un tipo de datos VARCHAR.|
|sortkey1_enc|	character (32)	|Codificación de compresión de la primera columna en la clave de ordenación, siempre que se haya definido una clave de ordenación.|
|sortkey_num	|entero	|Cantidad de columnas definidas como claves de ordenación.|
|size|	bigint|	Tamaño de la tabla, en bloques de datos de 1 MB.|
|pct_used	|numeric (10,4)|	Porcentaje de espacio disponible que usa la tabla.|
|empty	|bigint	|Para uso interno. Esta columna ya no se utiliza y se eliminará en una futura versión.|
|unsorted	|numeric (5,2)	|Porcentaje de filas desordenadas en la tabla.|
|stats_off	|numeric (5,2)|	Número que indica cuán obsoletas son las estadísticas de la tabla; 0 es actual y 100 es desactualizada.|
|tbl_rows|	numeric (38,0)	|Cantidad total de filas en la tabla. Este valor incluye las filas marcadas para ser eliminadas, pero que aún no se han limpiado.|
|skew_sortkey1	|numeric (19,2)|	Proporción del tamaño de la columna más grande que no sea una clave de ordenación con el tamaño de la primera columna de la clave de ordenación, siempre que se haya definido una clave de ordenación. Utilice este valor para evaluar la eficacia de la clave de ordenación.|
|skew_rows	|numeric (19,2)	|Proporción de la cantidad de filas en el sector con la mayor cantidad de filas con la cantidad de filas en el sector con la menor cantidad de filas.|
|estimated_visible_rows	|numeric (38,0)	|Las filas estimadas en la tabla. Este valor no incluye las filas marcadas para ser eliminadas.|
|risk_event|	texto|	Información de riesgo sobre una tabla. El campo está separado en partes: `risk_typex/id/timestamp` risk_type, donde 1 indica que se ejecutó COPY command with the EXPLICIT_IDS option. Amazon Redshift ya no verifica la unicidad de las columnas IDENTITY en la tabla. Para obtener más información, consulte EXPLICIT_IDS. El ID de la transacción, xid, que introduce el riesgo. El timestamp cuando se ejecutó el comando COPY. El siguiente ejemplo muestra el valor en el campo. `1/1107/2019-06-22 07:16:11.292952`|
|vacuum_sort_benefit	|numeric(12,2)|	El porcentaje máximo de mejora estimado del rendimiento de la consulta del escaneo cuando ejecuta una ordenación vacuum.
|create_time	|TIMESTAMP sin zona horaria	|Marca de tiempo UNIX de cuando se creó la tabla.|

```sql
-- Checkeamos la estructura con "svv_table_info"
SELECT *
FROM pg_catalog.svv_table_info
LIMIT 10;
```

![](img_72.png)

Filtramos por las tablas creadas anteriormente.

```sql
SELECT *
FROM pg_catalog.svv_table_info
WHERE "table" LIKE '%dist%'
LIMIT 10;
```

![](img_73.png)

### <a name="mark_14.0.0"></a>SVV_TABLE
[Index](#index_01)

Utilice SVV_TABLES para ver las tablas en catálogos locales y externos (es un resumen de SVV_TABLE_INFO).

SVV_TABLES es visible para todos los usuarios. Los superusuarios pueden ver todas las filas; los usuarios normales solo pueden ver los metadatos a los que tienen acceso.

#### Columnas de la tabla

|Nombre de la columna	|Tipo de datos	|Descripción|
|:---|:---:|:---|
|table_catalog|	texto	|Nombre del catálogo donde se encuentra la tabla.|
|table_schema|	texto	|Nombre del esquema para la tabla.|
|table_name|	texto	|El nombre de la tabla.|
|table_type|	texto	|El tipo de tabla. Los valores posibles son vistas, tablas externas y tablas base.|
|remarks	|texto	|Observaciones.|

### <a name="mark_14.1"></a>PG_TABLE_DEF, Almacena información acerca de las columnas de la tabla.
[Index](#index_01)

PG_TABLE_DEF solo devuelve información acerca de las tablas que son visibles para el usuario. Si PG_TABLE_DEF no devuelve los resultados esperados, verifique que el parámetro search_path se estableció correctamente para incluir los esquemas relevantes.

Puede utilizar [SVV_TABLE_INFO](https://docs.aws.amazon.com/es_es/redshift/latest/dg/r_SVV_TABLE_INFO.html) para consultar información más exhaustiva acerca de una tabla, incluidos el sesgo de distribución de datos, el sesgo de distribución de claves, el tamaño de tabla y las estadísticas.

#### Columnas de la tabla

|Nombre de la columna	|Tipo de datos	|Descripción|
|:---|:---:|:---|
|schemaname	|nombre	|Nombre del esquema.|
|tablename	|nombre	|Nombre de la tabla.|
|column	|nombre|	Nombre de la columna.|
|type	|texto	|Tipo de datos de la columna.|
|encoding	|character (32)	|Codificación de la columna.|
|distkey	|booleano	|True si esta columna es la clave de distribución para la tabla.|
|sortkey	|entero|	Orden de la columna en la clave de ordenación. Si la tabla utiliza una clave de ordenación compuesta, todas las columnas que forman parte de la clave de ordenación tienen un valor positivo que indica la posición de la columna en la clave de ordenación. Si la tabla utiliza una clave de ordenación intercalada, cada una de las columnas que formen parte de la clave de ordenación tienen un valor que puede ser positivo o negativo, donde el valor absoluto indica la posición de la columna en la clave de ordenación. Si es 0, la columna no forma parte de la clave de ordenación.|
|notnull|	booleano|	True si la columna tiene una restricción NOT NULL.|

```sql
SELECT *
FROM pg_table_def
WHERE tablename = 'users';
```
![](img_74.png)

Por ejemplo para la tabla "users" podemos identificar la columna utilizada para la distribution key.

![](img_75.png)

Me gustaría saber cómo están distribuidos mis datos.

### <a name="mark_14.2"></a>SVV_DISKUSAGE, combinación de las tablas STV_TBL_PERM y STV_BLOCKLIST
[Index](#index_01)

Amazon Redshift crea la vista de sistema SVV_DISKUSAGE mediante la combinación de las tablas STV_TBL_PERM y STV_BLOCKLIST. La vista SVV_DISKUSAGE tiene información relacionada con la asignación de datos para las tablas en una base de datos.

Utilice consultas de agregación con SVV_DISKUSAGE, ver [ejemplos](https://docs.aws.amazon.com/es_es/redshift/latest/dg/r_SVV_DISKUSAGE.html), para determinar la cantidad de bloques de disco asignados a cada base de datos, tabla, sector o columna. Cada bloque de datos utiliza 1 MB. También puede utilizar [STV_PARTITIONS](https://docs.aws.amazon.com/es_es/redshift/latest/dg/r_STV_PARTITIONS.html) para obtener información resumida acerca de la utilización del disco.

Solo los superusuarios pueden ver SVV_DISKUSAGE. Para obtener más información, consulte [Visibilidad de datos en las tablas y vistas de sistema](https://docs.aws.amazon.com/es_es/redshift/latest/dg/cm_chap_system-tables.html#c_visibility-of-data).

#### nota:
Esta vista solo está disponible cuando se consultan clústeres aprovisionados.

#### Columnas de la tabla

|Nombre de la columna|	Tipo de datos	|Descripción|
|:---|:---:|:---|
|db_id	|entero	|ID de la base de datos.|
|nombre|	character (72)|	Nombre de la tabla.|
|slice	|entero|	Sector de datos asignado a la tabla.|
|col	|entero	|Índice con base cero para la columna. Cada tabla que cree tiene tres columnas ocultas anexadas: INSERT_XID, DELETE_XID y ROW_ID (OID). Una tabla con 3 columnas definidas por el usuario tiene 6 columnas reales, y las columnas definidas por el usuario se enumeran internamente como 0, 1 y 2. Las columnas INSERT_XID, DELETE_XID y ROW_ID se enumeran 3, 4 y 5, respectivamente, en este ejemplo.
|tbl|	entero|	ID de la tabla.|
|blocknum|	entero	|ID para el bloque de datos.|
|num_values|	entero|	Cantidad de valores contenidos en el bloque.|
|minvalue|	bigint	|Valor mínimo contenido en el bloque.|
|maxvalue|	bigint|	Valor máximo contenido en el bloque.|
|sb_pos	|entero	|Identificador interno para la posición del super bloque en el disco.|
|pinned	|entero	|Indica si el bloque se conectó o no a la memoria como parte de la carga previa: 0 = false; 1 = true. La opción predeterminada es falso.|
|on_disk	|entero|	Indica si el bloque se almacenó automáticamente o no en el disco: 0 = false; 1 = true. La opción predeterminada es falso.|
|modified	|entero	|Indica si el bloque se modificó o no: 0 = false; 1 = true. La opción predeterminada es falso.
|hdr_modified|	entero	|Indica si el encabezado del bloque se modificó o no: 0 = false; 1 = true. La opción predeterminada es falso.|
|unsorted	|entero	|Indica si el bloque está desordenado o no: 0 = false; 1 = true. El valor predeterminado es verdadero.|
|tombstone	|entero	|Para uso interno.|
|preferred_diskno|	entero|	Cantidad de discos en que debe estar el bloque, excepto que el disco tenga una falla. Una vez arreglado el disco, el bloque volverá a ese disco.|
|temporary	|entero	|Indica si el bloque tiene o no datos temporales, como una tabla temporal o resultados intermedios de consulta: 0 = false; 1 = true. La opción predeterminada es falso.|
|newblock	|entero	|Indica si un bloque es o no nuevo (true) o si nunca se guardó en el disco (false): 0 = false; 1 = true.|
```sql
SELECT *
FROM pg_catalog.svv_diskusage 
LIMIT 10;
```

![](img_76.png)

Buscamos la tabla "users" y la columna que sabemos que es "distribution key".

```sql
SELECT *
FROM pg_catalog.svv_diskusage 
WHERE "name" = 'users'
AND col = 0;
```

![](img_77.png)

Veamos el comportamiento de los 4 slices y la distribución de los datos.

```sql
SELECT distinct slice, col, num_values, "minvalue", "maxvalue"
FROM pg_catalog.svv_diskusage 
WHERE "name" = 'users'
AND col = 0
AND num_values > 0
ORDER BY slice, col;
```

Como podemos ver los datos están muy bien distribuidos.

![](img_78.png)

La siguiente sentencia crea una nueva tabla "user_key_state", en la cual ahora su distribution key será la columna "state" usando/copiando la tabla "users".
```sql
CREATE TABLE user_key_state distkey(state) AS (SELECT * FROM users)
```

A continuación realizamos el mismo análisis pero con la nueva tabla "user_key_state"

```sql
SELECT distinct slice, col, num_values, "minvalue", "maxvalue"
FROM pg_catalog.svv_diskusage 
WHERE "name" = 'user_key_state'
AND col = 0
AND num_values > 0
ORDER BY slice, col;
```

A continuación podemos ver que la distribución por la columna "state" no fue muy performante.

![](img_79.png)

Repetimos el ejercicio para **"diststyle even"**:

```sql
CREATE TABLE user_even diststyle even AS (SELECT * FROM users)

SELECT distinct slice, col, num_values, "minvalue", "maxvalue"
FROM pg_catalog.svv_diskusage 
WHERE "name" = 'user_even'
AND col = 0
AND num_values > 0
ORDER BY slice, col;
```

Como vemos las columnas/datos están equitativamente distribuidos, pero recordar que están distribuidos al azar por el algoritomo RR, esto quiere decir que a pesar de que se encuentre perfectamente distribuido no será una distribución optima para realizar analítica.

![](img_80.png)

Probando con **"diststyle all"**

```sql
CREATE TABLE user_all diststyle all AS (SELECT * FROM users)

SELECT distinct slice, col, num_values, "minvalue", "maxvalue"
FROM pg_catalog.svv_diskusage 
WHERE "name" = 'user_all'
AND col = 0
AND num_values > 0
ORDER BY slice, col;

```

Como vemos está replicando cada tabla en el primer slice de cada nodo.

![](img_81.png)

### Lecturas Recomendadas:

https://docs.aws.amazon.com/es_es/redshift/latest/dg/r_SVV_TABLE_INFO.html

https://docs.aws.amazon.com/es_es/redshift/latest/dg/r_SVV_TABLES.html

https://docs.aws.amazon.com/es_es/redshift/latest/dg/r_PG_TABLE_DEF.html

https://docs.aws.amazon.com/es_es/redshift/latest/dg/r_CREATE_TABLE_examples.html

### <a name="mark_15"></a> sortkey, Llaves de ordenamiento para optimizar nuestras consultas

### [Index](#index_01)

Ordenar las columnas por cierto criterio para que sean guardadas en nuestros nodos, cuando evaluamos las tablas y columnas podemos ver una columna de valores mínimos y valores máximos, esto quiere decir que cada bloque de datos sabe en que rango de filas está trabajando, por ejemplo rango numerico, alfabetico, o de fechas, si utilizamos un "where date between ..." esta sentencia descartará automaticamente las fechas que no están en ese rango y solo buscará los bloques para los cuales el mínimo y máximo valor son útiles.

![](img_82.png)

#### nota:

Se recomienda que cree sus tablas con SORTKEY AUTO. En este caso, Amazon Redshift utiliza la optimización automática de tablas para elegir la clave de ordenación. Para obtener más información, consulte Uso de la optimización automática de tablas. En el resto de esta sección, se proporcionan detalles sobre el orden.

Cuando se crea una tabla, se pueden definir, como alternativa, una o varias de sus columnas como claves de ordenación. Cuando carga por primera vez los datos en una tabla vacía, las filas se almacenan en el disco de forma ordenada. Se transmite la información relacionada con las columnas con clave de ordenación al planificador de consultas, que utiliza esta información para construir planes que aprovechen la forma en la que se almacenan los datos. Para obtener más información, consulte [CREATE TABLE](https://docs.aws.amazon.com/es_es/redshift/latest/dg/r_CREATE_TABLE_NEW.html). Para obtener información sobre las prácticas recomendadas a la hora de crear una clave de clasificación, consulte [Elección de claves de ordenación recomendadas](https://docs.aws.amazon.com/es_es/redshift/latest/dg/c_best-practices-sort-key.html).

La ordenación permite encargarse eficazmente de predicados restringidos por rangos. Amazon Redshift almacena los datos de columna en bloques de disco de 1 MB. Los valores mínimo y máximo de cada bloque se almacenan como parte de los metadatos. Si una consulta usa un predicado de rango restringido, el procesador de consultas puede usar los valores mínimos y máximos para omitir rápidamente grandes cantidades de bloques durante los análisis de las tablas. Por ejemplo, supongamos que una tabla almacena cinco años de datos ordenados por fecha y una consulta especifica un rango de fechas correspondiente a un mes. En este caso, se puede eliminar hasta el 98 % de los bloques de disco del análisis. Si los datos no están ordenados, se deben examinar más bloques del disco (posiblemente todos).

Puede especificar una clave de ordenación compuesta o intercalada. Una clave de ordenación compuesta es más eficaz cuando los predicados de la consulta usan un prefijo, que es un subconjunto ordenado de columnas de clave de ordenación. Una clave de ordenación intercalada le otorga el mismo peso a cada columna de la clave de ordenación, por lo que los predicados de la consulta pueden usar cualquier subconjunto de columnas que conforman la clave de ordenación, en cualquier orden.

Para comprender el impacto de la clave de ordenación seleccionada en el rendimiento de las consultas, utilice el comando [EXPLAIN](https://docs.aws.amazon.com/es_es/redshift/latest/dg/r_EXPLAIN.html). Para obtener más información, consulte [Flujo de trabajo de planificación y ejecución de consultas](https://docs.aws.amazon.com/es_es/redshift/latest/dg/c-query-planning.html).

Para definir un tipo de ordenación, utilice la palabra clave INTERLEAVED o COMPOUND en su instrucción CREATE TABLE o CREATE TABLE AS. La opción predeterminada es COMPOUND. Se recomienda el COMPOUND cuando actualice las tablas a menudo con operaciones INSERT, UPDATE o DELETE. Una clave de ordenación INTERLEAVED puede usar como máximo ocho columnas. En función de los datos y del tamaño del clúster, VACUUM REINDEX necesita mucho más tiempo que VACUUM FULL ya que realiza una ejecución adicional para analizar las claves de ordenación intercaladas. La operación de clasificación y fusión tarda más tiempo para tablas intercaladas porque es posible que la clasificación intercalada tenga que reorganizar más filas que una clasificación compuesta.

Para ver las claves de ordenación de una tabla, consulte la vista de sistema [SVV_TABLE_INFO](https://docs.aws.amazon.com/es_es/redshift/latest/dg/r_SVV_TABLE_INFO.html).

![](img_83.png)

La "Simple" --> Es una llave COMPUOND pero por una sola columna.

#### Temas

Clave de ordenación compuesta (COMPOUND)
Clave de ordenación intercalada (INTERLEAVED)

### <a name="mark_15.0"></a>Clave de ordenación compuesta (COMPOUND)
[Index](#index_01)

![](img_84.png)

- Llave primaria --> "region".

- Llave secundaria --> "ciudad", "segmento", "fecha_creacion"

- Funciona muy bien si vamos a filtrar por todas las columnas compuestas.

- Muy mal si solo filtro por ciertas columnas del arreglo.

- Peor aún peor si filtro solo por columnas secundarias (ejemplo por una sola "ciudad").

Una clave compuesta está formada por todas las columnas presentes en la definición de la clave de ordenación, en el orden que aparecen. **Una clave de ordenación compuesta es más útil cuando el filtro de una consulta aplica condiciones, como filtros y combinaciones, que usan un prefijo de clave de ordenación. Los beneficios del rendimiento de la ordenación compuesta se reducen cuando las consultas dependen solo de las columnas de ordenación secundarias, sin hacer referencia a las columnas principales. COMPOUND es el tipo de ordenación predeterminado.**

Las claves de ordenación compuesta pueden acelerar las combinaciones, las operaciones GROUP BY y ORDER BY y las funciones de ventana que usan PARTITION BY y ORDER BY. Por ejemplo, una combinación de fusión, que suele ser más rápida que una combinación hash, es factible cuando los datos se distribuyen y se ordenan previamente en las columnas de combinación. Las claves de ordenación compuesta también ayudan a mejorar la compresión.

A medida que agrega filas a una tabla ordenada que ya tiene datos, la región desordenada aumenta y esto tiene un efecto significativo en el rendimiento. El efecto es mayor cuando la tabla usa una ordenación intercalada, en especial cuando las columnas de ordenación incluyen datos que aumentan de forma monótona, como las columnas de fecha o de marca temporal. Ejecute una operación VACUUM de manera periódica, sobre todo después de grandes cargas de datos, para volver a clasificar y a analizar los datos. Para obtener más información, consulte [Administración del tamaño de la región no ordenada](https://docs.aws.amazon.com/es_es/redshift/latest/dg/r_vacuum_diskspacereqs.html). Después hacer la limpieza para volver a ordenar los datos, se recomienda ejecutar un comando ANALYZE para actualizar los metadatos estadísticos para el planificador de consultas. Para obtener más información, consulte [Análisis de tablas](https://docs.aws.amazon.com/es_es/redshift/latest/dg/t_Analyzing_tables.html).

### <a name="mark_15.1"></a>Clave de ordenación intercalada (INTERLEAVED)
[Index](#index_01)

![](img_85.png)

![](img_86.png)

- Para este caso, la llave secundaria "ciudad" tiene la misma prioridad que la llave primaria "region"

- Si queremos optimizar cargas, no utilizar llaves intercaladas (INTERLEAVED)

Una clave de ordenación intercalada le otorga el mismo peso a cada columna o subconjunto de columnas en la clave de ordenación. Si hay distintas consultas que usan diferentes columnas para filtros, puede, por lo general, mejorar el rendimiento de esas consultas utilizando un estilo de ordenación intercalada. Cuando una consulta usa predicados restrictivos en las columnas de ordenación secundarias, la ordenación intercalada mejora considerablemente el rendimiento de la consulta, si se la compara con la ordenación compuesta.

#### Importante
No utilice una clave de ordenación intercalada con atributos monótonamente crecientes, como columnas de identidad, fechas o marcas temporales.

Las mejoras de rendimiento que obtiene al implementar una clave de ordenación intercalada se deben compensar con el aumento de los tiempos de carga y de limpieza.

Las ordenaciones intercaladas son más eficaces con las consultas sumamente selectivas que filtran una o más columnas con clave de ordenación en la cláusula WHERE; por ejemplo, la consulta select c_name from customer where c_region = 'ASIA'. Los beneficios de la ordenación intercalada aumentan con la cantidad de columnas ordenadas que están limitadas.

La ordenación intercalada es más eficaz con las tablas grandes. La ordenación se aplica a cada sector. Por lo tanto, una ordenación intercalada resulta más eficaz si una tabla es lo suficientemente grande como para requerir varios bloques de 1 MB por sector. En este caso, el procesador de consultas puede omitir una proporción significativa de los bloques mediante el uso de predicados restrictivos. Para ver la cantidad de bloques que utiliza una tabla, consulte la vista de sistema [STV_BLOCKLIST](https://docs.aws.amazon.com/es_es/redshift/latest/dg/r_STV_BLOCKLIST.html).

Cuando se ordena una sola columna, una ordenación intercalada puede tener un mejor rendimiento que una ordenación compuesta si los valores de la columna tienen un prefijo largo en común. Por ejemplo, los URL, por lo general, comienzan con "http://www". Las claves de ordenación compuesta usan una cantidad limitada de caracteres del prefijo, lo que genera muchas duplicaciones de claves. Las ordenaciones intercaladas utilizan un esquema de compresión interno para los valores de mapas de zonas que les permite discriminar mejor entre valores de columnas que tienen un prefijo largo en común.

Al migrar clústeres aprovisionados de Amazon Redshift a Amazon Redshift sin servidor, Redshift convierte tablas con claves de clasificación intercaladas y DISTSTYLE KEY en claves de clasificación compuestas. El DISTSTYLE no cambia. Para obtener más información sobre los estilos de distribución, consulte [Uso de estilos de distribución de datos](https://docs.aws.amazon.com/es_es/redshift/latest/dg/t_Distributing_data.html).

### VACUUM REINDEX

A medida que agrega filas a una tabla ordenada que ya tiene datos, el rendimiento puede deteriorarse con el tiempo. Este deterioro ocurre tanto en las ordenaciones compuestas como en las intercaladas, pero tiene mayor efecto en las tablas intercaladas. El comando VACUUM restaura el orden, pero la operación puede tomar más tiempo para las tablas intercaladas porque fusionar datos nuevos intercalados puede implicar modificar cada bloque de datos.

Cuando las tablas se cargan por primera vez, Amazon Redshift analiza la distribución de los valores de las columnas con clave de ordenación y usa la información para intercalar de manera óptima las columnas con clave de ordenación. A medida que una tabla aumenta, la distribución de los valores de las columnas con clave de ordenación puede variar o generar un sesgo, en especial con las columnas de fecha o de marca temporal. Si el sesgo es demasiado grande, puede afectar el rendimiento. Para volver a analizar las claves de ordenación y restaurar el rendimiento, ejecute el comando VACUUM con la palabra clave REINDEX. Como el comando VACUUM REINDEX debe hacer un análisis adicional sobre los datos, puede tardar más que un comando VACUUM estándar para las tablas intercaladas. Para ver información acerca del sesgo de distribución de claves y del tiempo de la última reindexación, consulte la vista de sistema [SVV_INTERLEAVED_COLUMNS](https://docs.aws.amazon.com/es_es/redshift/latest/dg/r_SVV_INTERLEAVED_COLUMNS.html).

Para obtener más información acerca de cómo determinar con qué frecuencia ejecutar el comando VACUUM y cuándo ejecutar el comando VACUUM REINDEX, consulte[Decidir si reindexar](https://docs.aws.amazon.com/es_es/redshift/latest/dg/r_vacuum-decide-whether-to-reindex.html).

### Lecturas Recomendadas:

https://docs.aws.amazon.com/es_es/redshift/latest/dg/t_Sorting_data.html

https://docs.aws.amazon.com/es_es/redshift/latest/dg/t_Creating_tables.html

https://docs.aws.amazon.com/es_es/redshift/latest/dg/r_CREATE_TABLE_examples.html

### <a name="mark_16"></a>Evaluando algoritmos de ordenamiento

### [Index](#index_01)

Para realizar este análisis se realizaron previamente algunas cuestiones, para generar el contexto de la prueba:

- Creación de una tabla llamada "cust_sales_date" vacía, sin ningún sort, diststyle, o compresión.
- Migración/populación utilizando "copy" y el archivo comprimido "cust_sales_date.bz2" del [repo](https://github.com/alarcon7a/redshift_course/tree/master/Claves_de_ordenamiento), observar que solo basta cargar el archivo comprimido en S3 y realizar un copy desde el bucket correspondiente.
    3. 

```sql

CREATE TABLE public.cust_sales_date (
	c_custkey int4 NULL,
	c_nation varchar(15) NULL,
	c_region varchar(12) NULL,
	c_mktsegment varchar(10) NULL,
	d_date date NULL,
	lo_revenue int4 NULL
);

copy cust_sales_date from 's3://mybucketredshift/cust_sales_date.bz2' 
credentials 'aws_iam_role=arn:aws:iam::XXXX:role/MiRoleRedshift' 
BZIP2
region 'us-east-2';
```
Checkeamos la cantidad de registros insertados "10 millones de registros".

```sql
SELECT COUNT(0) FROM cust_sales_date; 
SELECT * FROM  cust_sales_date LIMIT 10;
```

![](img_87.png)

![](img_88.png)


- Creación de tabla "auxiliar" que tiene 1 columna y 5 filas, el objetivo de esta tabla es solo para hacer un "producto cartesiano" con la tabla "cust_sales_date" y llevar los registros de 10 millones a 50 millones.

```sql

CREATE TABLE auxiliar (col int);
INSERT INTO auxiliar VALUES (1), (2), (3), (4), (5);
SELECT * FROM auxiliar;

```
![](img_89.png)


- Ahora crearemos y poblaremos (cada tabla con una misma sentencia SELECT) las 3 tablas con sus diferentes sortkey (simple/automatico, compound, y interleaved) para el análisis de su comportamiento.

```sql

-- SIMPLE
CREATE TABLE cust_sales_simple
sortkey (c_custkey)
AS (
	SELECT c_custkey, c_nation, c_region, c_mktsegment, d_date, lo_revenue
	FROM cust_sales_date, auxiliar
	);

-- COMPOUND
CREATE TABLE cust_sales_compuesto
compound sortkey (c_custkey, c_region, c_mktsegment, d_date)
AS (
	SELECT c_custkey, c_nation, c_region, c_mktsegment, d_date, lo_revenue
	FROM cust_sales_date, auxiliar
	);

-- INTERLEAVED
CREATE TABLE cust_sales_intercalado
interleaved sortkey (c_custkey, c_region, c_mktsegment, d_date)
AS (
	SELECT c_custkey, c_nation, c_region, c_mktsegment, d_date, lo_revenue
	FROM cust_sales_date, auxiliar
	);
    
SELECT count(0) FROM cust_sales_simple;	
SELECT count(0) FROM cust_sales_compuesto;
SELECT count(0) FROM cust_sales_intercalado;

```

En este punto cada tabla con exactamente los mismos datos se encuentra con sus respectivas llaves de ordenamiento, procedemos a realizar la misma consulta para cada tabla y evaluar el tiempo de ejecución, ingresando por su llave primaria "c_custkey".

### <a name="test_01"></a>`*********************************PRIMER PRUEBA********************************************`
[Index](#index_01)

### Simple primer análisis prueba 01:

```sql

SELECT max(lo_revenue), min(lo_revenue) 
FROM cust_sales_simple
WHERE c_custkey < 100000;
```

Tiempo de ejecución --> **4.226s**

![](img_90.png)

### Compound primer análisis prueba 01:

```sql
SELECT max(lo_revenue), min(lo_revenue) 
FROM cust_sales_compuesto
WHERE c_custkey < 100000;
```

Tiempo de ejecución --> **118ms**

![](img_91.png)

### Interleaved primer análisis prueba 01:

```sql
SELECT max(lo_revenue), min(lo_revenue) 
FROM cust_sales_intercalado
WHERE c_custkey < 100000;

```

Tiempo de ejecución --> **3.690s**

![](img_92.png)

Hasta el momento siendo estas las primeras ejecuciones que tiene Redshift sobre las tablas la mejor performance la tiene "Compound", para evitar el sesgo que el guardado en cache nos ocacionarían para repetidas consultar de una misma tabla, realizamos el apagado de cache y medimos nuevamente las 3 consultas.

```sql

set enable_result_cache_for_session to off;

```

### Simple primer análisis prueba 02:

![](img_93.png)

Tiempo de ejecución --> **189ms**

### Compound primer análisis prueba 02:

![](img_94.png)

Tiempo de ejecución --> **123ms**

### Interleaved primer análisis prueba 02:

![](img_95.png)

Tiempo de ejecución --> **178ms**

#### Tabla de comparación

|simple|compound|interleaved|comments|
|:---:|:---:|:---:|:---|
|4.226s|118ms|3.690s|prueba 01|
|189ms|123ms|178ms|prueba 02|

Como podemos ver "compound", sigue siendo, en este contexto, la mejor, provemos con diferentes consultas.

### <a name="test_02"></a>`*********************************SEGUNDA PRUEBA********************************************`
[Index](#index_01)

### Simple segundo análisis prueba 01
```sql

SELECT max(lo_revenue), min(lo_revenue) 
FROM cust_sales_simple
WHERE c_region = 'ASIA'
AND c_mktsegment = 'FURNITURE'; 
```
![](img_96.png)

Tiempo de ejecución --> **4.374s**


### Compound segundo análisis prueba 01
```sql
SELECT max(lo_revenue), min(lo_revenue) 
FROM cust_sales_compuesto
WHERE c_region = 'ASIA'
AND c_mktsegment = 'FURNITURE';
```

![](img_97.png)

Tiempo de ejecución --> **4.450s**

### Interleaved segundo análisis prueba 01

```sql
SELECT max(lo_revenue), min(lo_revenue) 
FROM cust_sales_intercalado
WHERE c_region = 'ASIA'
AND c_mktsegment = 'FURNITURE';

```

![](img_98.png)

Tiempo de ejecución --> **3.856s**

Apagamos nuevamente el cache y repetimos.

```sql

set enable_result_cache_for_session to off;

```

### Simple segundo análisis prueba 02:

![](img_99.png)

Tiempo de ejecución --> **612ms**

### Compound segundo análisis prueba 02:

![](img_100.png)

Tiempo de ejecución --> **361ms**

### Interleaved segundo análisis prueba 02:

![](img_101.png)

Tiempo de ejecución --> **164ms**

#### Tabla de comparación

|simple|compound|interleaved|comments|
|:---:|:---:|:---:|:---|
|4.374s|4.450s|3.856s|prueba 01|
|612ms|361ms|164ms|prueba 02|

El ordenamiento intercalado, para este caso, al tener la misma prioridad para cada llave es más eficiente en la búsqueda.

### <a name="test_03"></a>`*********************************TERCER PRUEBA*********************************`
[Index](#index_01)

### Simple tercer análisis prueba 01
```sql
SELECT max(lo_revenue), min(lo_revenue)
FROM cust_sales_simple
WHERE d_date BETWEEN '01/01/1996' AND '01/14/1996'
AND c_mktsegment = 'FURNITURE'
AND c_region = 'ASIA'; 
```
![](img_102.png)

Tiempo de ejecución --> **5.44s**


### Compound tercer análisis prueba 01
```sql
SELECT max(lo_revenue), min(lo_revenue)
FROM cust_sales_compuesto
WHERE d_date BETWEEN '01/01/1996' AND '01/14/1996'
AND c_mktsegment = 'FURNITURE'
AND c_region = 'ASIA';
```

![](img_103.png)

Tiempo de ejecución --> **5.395s**

### Interleaved tercer análisis prueba 01

```sql
SELECT max(lo_revenue), min(lo_revenue)
FROM cust_sales_intercalado
WHERE d_date BETWEEN '01/01/1996' AND '01/14/1996'
AND c_mktsegment = 'FURNITURE'
AND c_region = 'ASIA';
```

![](img_104.png)

Tiempo de ejecución --> **5.499s**

Apagamos nuevamente el cache y repetimos.

```sql

set enable_result_cache_for_session to off;

```

### Simple tercer análisis prueba 02:

![](img_105.png)

Tiempo de ejecución --> **680ms**

### Compound tercer análisis prueba 02:

![](img_106.png)

Tiempo de ejecución --> **432ms**

### Interleaved tercer análisis prueba 02:

![](img_107.png)

Tiempo de ejecución --> **143ms**

#### Tabla de comparación

|simple|compound|interleaved|comments|
|:---:|:---:|:---:|:---|
|5.44s|5.395s|5.499s|Prueba 01|
|680ms|432ms|143ms|Prueba 02|

El ordenamiento intercalado, para este caso, al tener la misma prioridad para cada llave es más eficiente en la búsqueda.

### <a name="conclusion_01"></a>Conclusiones:

### [Primer prueba](#test_01)

#### Utilizando la llave primaria para las consultas "COMPOUND" fue más eficiente.

|simple|compound|interleaved|comments|
|:---:|:---:|:---:|:---|
|4.226s|118ms|3.690s|prueba 01|
|189ms|123ms|178ms|prueba 02|

### [Segunda prueba](#test_02)

#### Utilizando distintans llaves secundarias, "INTERLEAVED" fue más eficiente.

|simple|compound|interleaved|comments|
|:---:|:---:|:---:|:---|
|4.374s|4.450s|3.856s|prueba 01|
|612ms|361ms|164ms|prueba 02|

### [Tercer prueba](#test_03)

#### Utilizando distintans llaves secundarias + un contexto limitado de fechas, "INTERLEAVED" fue más eficiente.

|simple|compound|interleaved|comments|
|:---:|:---:|:---:|:---|
|5.44s|5.395s|5.499s|Prueba 01|
|680ms|432ms|143ms|Prueba 02|

Si bien una tabla con ordenamiento intercalado tarda más en ser populada y su mantenimiento es un extra administrativo que debemos tener en cuenta, al momento de las consultas es mucho más eficiente, en 2 de las 3 consultas, como lo demuestran los análisis en cada contexto.

### Observación:

Explicando el porque de las diferencias temporales entre la Prueba_01 y la Prueba_02.

En la primer ejecución de cada prueba, el cache no estaba apagado, entonces Redshift analiza primero si tiene ya ese resultado y lo intenta mostrar, por lo cual es más lento. Pero en cada segunda prueba este cache se apago entonces simplemente ejecuta la centencia directamente, por eso para el ejercicio es más rápido.

### <a name="mark_17"></a> Buenas prácticas para diseñar tablas en Redshift

### [Index](#index_01)

Si bien los constrains no son parte esencias de una db (datalake/datawarehouse/datalakehouse) para analítica, en Redshift si podemos crear esta integridad será mucho mejor, ya que la velocidad de consulta será mayor.

![](img_108.png)

- Siempre utiliza o transforma las fechas a un tipo de formato.
- En lo posible limitar la longitud de los campos (varchar) al número más exactamente necesario, esto mejora los algoritmos de compresión en Redshift.
- Crear comentarios explicativos sobre lo que sea necesario.

![](img_109.png)

La siguiente query es una query que se encontraba en la Documentación (actualmente no es posible encontrarla como "análisis del diseño de tablas"). Que muestra las siguientes columnas:

_ schemaname

_ tablename

_ tableid

_ size_in_mb

_ has_disk_key --> tiene o no una llave de distribución

_ has_sort_key --> tiene o no una llave de ordenamiento

_ has_col_encoding --> tiene o no columnas comprimidas

_ ratio_skew_across_slices --> indica si hay un sesgo en la distribución de los datos (un valor bajo significa algo bueno)

_ pct_slices_populated --> muestra en % cuán lleno de datos están los slices en una tabla (un valor del 100% es bueno)
```sql

select
	schema schemaname,
	"table" tablename,
	table_id tableid,
	size size_in_mb,
	case
		when diststyle not in ('EVEN', 'ALL') then 1
		else 0
	end has_dist_key,
	case
		when sortkey1 is not null then 1
		else 0
	end has_sort_key,
	case
		when encoded = 'Y' then 1
		else 0
	end has_col_encoding,
	cast(max_blocks_per_slice - min_blocks_per_slice as FLOAT) / greatest(NVL (min_blocks_per_slice, 0)::int, 1) ratio_skew_across_slices,
	cast(100 * dist_slice as FLOAT) /(
	select
		COUNT(distinct slice)
	from
		stv_slices) pct_slices_populated
from
	svv_table_info ti
join (
	select
		tbl, MIN(c) min_blocks_per_slice, MAX(c) max_blocks_per_slice, COUNT(distinct slice) dist_slice
	from
		(
		select
			b.tbl, b.slice, COUNT(*) as c
		from
			STV_BLOCKLIST b
		group by
			b.tbl, b.slice)
	where
		tbl in (
		selectselect
	schema schemaname,
	"table" tablename,
	table_id tableid,
	size size_in_mb,
	case
		when diststyle not in ('EVEN', 'ALL') then 1
		else 0
	end has_dist_key,
	case
		when sortkey1 is not null then 1
		else 0
	end has_sort_key,
	case
		when encoded = 'Y' then 1
		else 0
	end has_col_encoding,
	cast(max_blocks_per_slice - min_blocks_per_slice as FLOAT) / greatest(NVL (min_blocks_per_slice, 0)::int, 1) ratio_skew_across_slices,
	cast(100 * dist_slice as FLOAT) /(
	select
		COUNT(distinct slice)
	from
		stv_slices) pct_slices_populated
from
	svv_table_info ti
join (
	select
		tbl, MIN(c) min_blocks_per_slice, MAX(c) max_blocks_per_slice, COUNT(distinct slice) dist_slice
	from
		(
		select
			b.tbl, b.slice, COUNT(*) as c
		from
			STV_BLOCKLIST b
		group by
			b.tbl, b.slice)
	where
		tbl in (
		select
			table_id
		from
			svv_table_info)
	group by
		tbl) iq on
	iq.tbl = ti.table_id;
			table_id
		from
			svv_table_info)
	group by
		tbl) iq on
	iq.tbl = ti.table_id;

```

### Lecturas Recomendadas:

https://docs.aws.amazon.com/es_es/redshift/latest/dg/c_analyzing-table-design.html


### <a name="mark_18"></a>Tipos de datos en AWS Redshift

### [Index](#index_01)

En esta clase conoceremos los tipos de datos que maneja Redshift, y como ya lo sabes, Redshift está basado en PostgreSQL, de manera que si estás familiarizado con los tipos de datos de PostgreSQL esta clase se te facilitará.

Como otras bases de datos, Redshift maneja datos de tipo numérico, texto, de lógica booleana y estos datos tienen ciertas particularidades.

### Tipos de datos para texto:

En estos tipos de datos almacenaremos caracteres y cadenas de texto como pueden ser nombres, direcciones, descripciones, etc.

Lo más importante que debes saber al momento de manejar estos tipos de datos es que el parámetro que le indiques a cada dato no es la longitud, es el número de bytes que trabajará, de manera que un dato VARCHAR(2) no indica que reciba 2 caracteres de longitud, sino que el contenido es de 2 bytes.

```sql

CREATE  TABLE  test_tipo_dato (
dato  varchar(2)
)

insert  into  test_tipo_dato  values ('AA');
insert  into  test_tipo_dato  values ('€');

```

Si ejecutas las anteriores sentencias notarás que el primer insert se ejecuta sin ningún problema, “A” es un carácter que solo ocupa 1 byte. De modo que “AA” ocupa un total de 2 bytes. Pero, la segunda sentencia de insert falló y esta solo contiene un carácter “€”, esto ocurre por que este carácter tiene una longitud de 3 bytes por sí solo, de manera que rompe la restricción de tipo de dato VARCHAR(2).

![](img_110.png)

Como ves los tipos de dato BPCHAR son convertidos por Redshift a CHAR y los tipos de dato TEXT a VARCHAR; también es importante tener en cuenta que un dato CHAR de 10, siempre ocupara 10 bytes, y dado el caso que el dato guardado no complete los 10 bytes Redshift lo completa con espacios en blanco, esto no ocurre en el tipo de dato VARCHAR.

Compruébalo tú mismo:

```sql

create  table  test_char (
dato  char(10)
);

insert  into  test_char  values ('dato');
select * from  test_char;

```

### Tipos de datos numéricos:

Estos tipos de datos se deben usar para el manejo de números, en casos como cantidades, medidas, llaves únicas etc, los datos numéricos pueden ser enteros, decimales y de coma flotante.

### Enteros:

![](img_111.png)

### Decimales:

![](img_112.png)

Para el manejo de datos decimales, decimal(precisión, escala), debes tener en cuenta que se manejan dos parámetros, la precisión y la escala.

Para explicarte como funciona usemos un ejemplo:

```sql

create  table  test_decimal (
dato  decimal(4,2)
);

insert  into  test_decimal  values (4581.54);
insert  into  test_decimal  values (121.7);
insert  into  test_decimal  values (12.21);
insert  into  test_decimal  values (12.2);
insert  into  test_decimal  values (12.21222);

```

Como puedes observar los dos primeros “insert” fallaron, y quizá te estés preguntando, ¿por qué ocurrió esto? si en la sentencia dice 4 enteros y 2 decimales… pero no, el valor de precisión (4 para este ejemplo) indica el número total de dígitos que puede recibir, y escala indica el número de decimales que se trabajarán, de modo que en realidad este dato puede máximo guardar un número de 2 enteros y 2 decimales (4,2), pero ¿y la última sentencia insert que tenía 2 enteros y 5 decimales? ¿por qué funcioné?, bueno Redshift nota que el número de enteros es válido y solo redondea las cifras decimales a 2 dígitos para este caso.

### Otros tipos de dato:

Los valores de fecha como date, timestamp, timestampz se manejan como cualquier otra base de datos, también los tipos de dato bool, boolean, los cuales solo pueden recibir un byte de longitud.

A continuación te comparto la tabla que usa Redshift para el manejo de tipos de datos.

![](img_113.png)


### <a name="mark_19"></a>Reto: mejora el desempeño de tu base de datos

### [Index](#index_01)

En linea con el diseño "estrella", Creamos las tablas...

_ part (dimensión)

_ supplier (dimensión)

_ customer (dimensión)

_ dwdate (dimensión)

_ lineorder(tabla de hechos)

Sin ningún tipo de compresión, ordenamiento, distribución, ni constrains.

```sql

CREATE TABLE part 
(
  p_partkey     INTEGER NOT NULL,
  p_name        VARCHAR(22) NOT NULL,
  p_mfgr        VARCHAR(6) NOT NULL,
  p_category    VARCHAR(7) NOT NULL,
  p_brand1      VARCHAR(9) NOT NULL,
  p_color       VARCHAR(11) NOT NULL,
  p_type        VARCHAR(25) NOT NULL,
  p_size        INTEGER NOT NULL,
  p_container   VARCHAR(10) NOT NULL
);

CREATE TABLE supplier 
(
  s_suppkey   INTEGER NOT NULL,
  s_name      VARCHAR(25) NOT NULL,
  s_address   VARCHAR(25) NOT NULL,
  s_city      VARCHAR(10) NOT NULL,
  s_nation    VARCHAR(15) NOT NULL,
  s_region    VARCHAR(12) NOT NULL,
  s_phone     VARCHAR(15) NOT NULL
);

CREATE TABLE customer 
(
  c_custkey      INTEGER NOT NULL,
  c_name         VARCHAR(25) NOT NULL,
  c_address      VARCHAR(25) NOT NULL,
  c_city         VARCHAR(10) NOT NULL,
  c_nation       VARCHAR(15) NOT NULL,
  c_region       VARCHAR(12) NOT NULL,
  c_phone        VARCHAR(15) NOT NULL,
  c_mktsegment   VARCHAR(10) NOT NULL
);

CREATE TABLE dwdate 
(
  d_datekey            INTEGER NOT NULL,
  d_date               VARCHAR(19) NOT NULL,
  d_dayofweek          VARCHAR(10) NOT NULL,
  d_month              VARCHAR(10) NOT NULL,
  d_year               INTEGER NOT NULL,
  d_yearmonthnum       INTEGER NOT NULL,
  d_yearmonth          VARCHAR(8) NOT NULL,
  d_daynuminweek       INTEGER NOT NULL,
  d_daynuminmonth      INTEGER NOT NULL,
  d_daynuminyear       INTEGER NOT NULL,
  d_monthnuminyear     INTEGER NOT NULL,
  d_weeknuminyear      INTEGER NOT NULL,
  d_sellingseason      VARCHAR(13) NOT NULL,
  d_lastdayinweekfl    VARCHAR(1) NOT NULL,
  d_lastdayinmonthfl   VARCHAR(1) NOT NULL,
  d_holidayfl          VARCHAR(1) NOT NULL,
  d_weekdayfl          VARCHAR(1) NOT NULL
);
CREATE TABLE lineorder 
(
  lo_orderkey          INTEGER NOT NULL,
  lo_linenumber        INTEGER NOT NULL,
  lo_custkey           INTEGER NOT NULL,
  lo_partkey           INTEGER NOT NULL,
  lo_suppkey           INTEGER NOT NULL,
  lo_orderdate         INTEGER NOT NULL,
  lo_orderpriority     VARCHAR(15) NOT NULL,
  lo_shippriority      VARCHAR(1) NOT NULL,
  lo_quantity          INTEGER NOT NULL,
  lo_extendedprice     INTEGER NOT NULL,
  lo_ordertotalprice   INTEGER NOT NULL,
  lo_discount          INTEGER NOT NULL,
  lo_revenue           INTEGER NOT NULL,
  lo_supplycost        INTEGER NOT NULL,
  lo_tax               INTEGER NOT NULL,
  lo_commitdate        INTEGER NOT NULL,
  lo_shipmode          VARCHAR(10) NOT NULL
);

```

Populando de datos las tablas:

- "awssampledbuswest2" es una carpeta oculta, y por defecto, en AWS que contiene estos ejemplos

- Reemplezar ARN del Role y la región correspondiente.

- "gzip compupdate off" No quiero comprimir los datos luego de la ingesta.

```sql

copy customer from 's3://awssampledbuswest2/ssbgz/customer' 
credentials 'aws_iam_role=<Reemplazar_con_tu_iam_role_arn>'  
gzip compupdate off region 'us-west-2';

copy dwdate from 's3://awssampledbuswest2/ssbgz/dwdate' 
credentials 'aws_iam_role=<Reemplazar_con_tu_iam_role_arn>'  
gzip compupdate off region 'us-west-2';

copy lineorder from 's3://awssampledbuswest2/ssbgz/lineorder' 
credentials 'aws_iam_role=<Reemplazar_con_tu_iam_role_arn>'  
gzip compupdate off region 'us-west-2';

copy part from 's3://awssampledbuswest2/ssbgz/part' 
credentials 'aws_iam_role=<Reemplazar_con_tu_iam_role_arn>'  
gzip compupdate off region 'us-west-2';

copy supplier from 's3://awssampledbuswest2/ssbgz/supplier' 
credentials 'aws_iam_role=<Reemplazar_con_tu_iam_role_arn>'  
gzip compupdate off region 'us-west-2';

```

Chequemos la creación de la tabla "customer"

![](img_114.png)

Analizamos la recomendación de compresión/encoding utilizando `analyze compression nombre_tabla`

![](img_115.png)

Aplicamos la recomendación creando una nueva tabla "customer_pro", notar que colocando la distribución "distkey" en la línea de la columna la declara automaticamente.

```sql

CREATE TABLE IF NOT EXISTS  public.customer_pro 
(
  c_custkey      INTEGER NOT NULL ENCODE az64 distkey,
  c_name         VARCHAR(25) NOT NULL ENCODE zstd,
  c_address      VARCHAR(25) NOT NULL ENCODE zstd,
  c_city         VARCHAR(10) NOT NULL ENCODE bytedict,
  c_nation       VARCHAR(15) NOT NULL ENCODE bytedict,
  c_region       VARCHAR(12) NOT NULL ENCODE bytedict,
  c_phone        VARCHAR(15) NOT NULL ENCODE zstd,
  c_mktsegment   VARCHAR(10) NOT NULL ENCODE bytedict
);

```

Poblemos la tabla ...

```sql

insert into customer_pro (select * from customer);

```

## Realizar el mismo procedimiento de optimización con las demás tablas y probar la eficacia con las siguientes queries.

```sql

--Utiliza la siguente sentencia para desactivar la cache de las consultas con esto buscamos no inteferir en la medicion de tiempos de respuesta
set enable_result_cache_for_session to off;

-- Query 1
select sum(lo_extendedprice*lo_discount) as revenue
from lineorder, dwdate
where lo_orderdate = d_datekey
and d_year = 1997
and lo_discount between 1 and 3
and lo_quantity < 24;

-- Query 2
select sum(lo_revenue), d_year, p_brand1
from lineorder, dwdate, part, supplier
where lo_orderdate = d_datekey
and lo_partkey = p_partkey
and lo_suppkey = s_suppkey
and p_category = 'MFGR#12'
and s_region = 'AMERICA'
group by d_year, p_brand1
order by d_year, p_brand1;

-- Query 3
select c_city, s_city, d_year, sum(lo_revenue) as revenue
from customer, lineorder, supplier, dwdate
where lo_custkey = c_custkey
and lo_suppkey = s_suppkey
and lo_orderdate = d_datekey
and (c_city='UNITED KI1' or
c_city='UNITED KI5')
and (s_city='UNITED KI1' or
s_city='UNITED KI5')
and d_yearmonth = 'Dec1997'
group by c_city, s_city, d_year
order by d_year asc, revenue desc;  

-- Query 4
select sum(lo_revenue) , c.c_nation from lineorder l 
inner join customer c 
on l.lo_custkey = c.c_custkey 
where c_region  = 'EUROPE'
group by c.c_nation
order by 1 desc

```

### Lectura Recomendada:

https://docs.aws.amazon.com/redshift/latest/dg/tutorial-tuning-tables-create-test-data.html

https://github.com/alarcon7a/redshift_course/blob/master/Reto_Dise%C3%B1o_Tablas/Sentencias.sql



### <a name="mark_20"></a>Olvídate de los insert, el copy llego para quedarse

### [Index](#index_02)

- Es mejor realizar la carga de multiples archivos en una sola ejecución que realizarlos uno por uno.

- Se pueden cargar archivos comprimidos.

![](img_116.png)


### Que debo tener en cuenta al momento de hacer un COPY.

- Otorgar permisos, lo hacemos con el rol de IAM.

- Recordar que la codificación original debe estar en UTF-8

- Particionar los datos para mejorar la performance de carga.

![](img_117.png)

![](img_118.png)


### Lecturas Recomendadas:

https://docs.aws.amazon.com/es_es/redshift/latest/dg/r_COPY.html

### <a name="mark_21"></a>Cargando archivos tipo JSON

### [Index](#index_02)

Ahora veremos cómo llevar nuestros datos a Redshift dado el caso que nuestro origen de datos se encuentre en un formato json.

Lo primero que debemos saber es que existen dos formas de cargar los datos en formato json en Redshift, una es la forma automática, en donde cada “key” del archivo json debe hacer match con cada columna que queramos llenar de una tabla en Redshift.

Es importante saber que las columnas en Redshift se crean en minúsculas, de manera que las “key” del archivo en json deben estar en minúscula también. La segunda forma es especificando la estructura del archivo para que este pueda ser cargado a la tabla, para ello utilizaremos un archivo extra denominado jsonpaths.

### Carga automática:

Usando la estructura de tabla que hemos manejado en este curso:

```sql
--Create table
create  table  estudiante
( id  int2,
nombre  varchar(20),
apellido  varchar(20),
edad  int2,
fecha_ingreso  date );
```

Crearemos una estructura en Redshift que pueda satisfacer las necesidades de esta tabla, de modo que puede quedar algo así (recuerda que los “key” del archivo se debe llamar igual que en la tabla):

```json

{
	"id": 4544,
	"nombre": "test_1",
	"apellido": "json_1",
	"edad": 33,
	"fecha_ingreso": "2020-08-01"
}
{
	"id": 23232,
	"nombre": "test_2",
	"apellido": "json_2",
	"edad": 22,
	"fecha_ingreso": "2020-08-03"
}

```

Ahora lo subimos a S3 con el nombre que queramos y procedemos a cargarlo de la siguiente manera:

```sql

copy  public.estudiante  from  's3://[tu_propia_ruta_del_archivo_json]'  credentials  'aws_iam_role=[tu_iam_role]'
format  as  json  'auto'  region  '[tu_region]';

```

### Carga con jsonpaths:

Esta carga consiste en determinar una estructura basada en el archivo json que pueda ser insertada en la tabla, como sabemos en un archivo json podemos tener datos no estructurados, pero Redshift requiere de data estructurada; de manera que lo primero será crear un pequeño ejemplo de un archivo json con datos no estructurada.

```json

{
	"id": 4544,
	"nombre": "test_json3",
	"apellido": "test_json3",
	"edad": [
		24332,
		33,
		443,
		323232,
		43434
	],
	"fechas": [
		{
			"id": 0,
			"fecha_ingreso": "2015-05-01"
		},
		{
			"id": 1,
			"fecha_ingreso": "2016-05-30"
		}
	]
}

```

Vamos a suponer que de la lista de edades son la segunda posición o sea la posición [1] es la que yo requiero, y para las fechas, la fecha que en verdad me interesa es la contenida en el segundo bloque, para especificar estas reglas crearemos un nuevo archivo con la siguiente estructura y lo nombraremos “jsonpaths”:

```json

{
	"jsonpaths": [
		"$['id']",
		"$['nombre']",
		"$['apellido']",
		"$['edad'][1]",
		"$['fechas'][1]['fecha_ingreso']"
	]
}

```
`*************************************************************************************************************************`

El símbolo $ se utiliza para acceder a los valores de un objeto JSON. En este caso, el objeto JSON se llama "jsonpaths" y tiene cinco propiedades: "id", "nombre", "apellido", "edad" y "fechas".

Las cadenas que comienzan con $ representan rutas JSON. Una ruta JSON es una secuencia de nombres de propiedades que se utilizan para acceder a un valor específico en un objeto JSON.

`*************************************************************************************************************************`

Ahora subimos nuestros dos archivos a S3 y ejecutamos la siguiente sentencia en nuestro editor SQL.

```sql

copy  public.estudiante  from  's3://[tu_propia_ruta_del_archivo_json]'  credentials  'aws_iam_role=[tu_iam_role]'
format  as  json  's3://[tu_propia_ruta_del_archivo_jsonpaths]'  region  'tu_propia_region';

```

Y como vemos, el registro fue cargado exitosamente.

![](img_119.png)

### <a name="mark_22"></a>El comando copy a fondo

### [Index](#index_02)

Para este ejemplo utilizaremos 3 archivos csv con diferentes datos.

### primer_cargue.csv

```csv

Id;Nombre;Apellido;Edad;Fecha_registro
1;Carlos;Alarcon;27;2020-06-15

2;Andres;Perez;45;2019-01-01
3;Felipe;Lopez;20;2020-07-03
4;  ;Garcia;35;2020-01-03
5;;Torres;14;2020-03-07

```

### segundo_cargue.csv

```csv

Id;Nombre;Apellido;Edad;Fecha_registro
1;Carlos;Alarcon;27;06-05-2020
2;Andres;Perez;45;06-05-2020
3;Felipe;Lopez;20;06-05-2020

```

### tercer_cargue.csv

```csv

1Carlos   Alarcon  2706-05-2020
2Andres   Perez    4506-05-2020
3Felipe   Lopez    2006-05-2020

```

Comenzamos a crear las tablas para popular los datos.

```sql

CREATE TABLE estudiante (
id int2,
nombre varchar(20),
apellido varchar(20),
edad int2,
fecha_ingreso date
);

```
Realizamos el primer intento de carga.

![](img_120.png)

Como vemos la carga falló, y el warning nos indica verificar la tabla "stl_load_errors", podemos ver que no hubo delimitador predefinido.

![](img_121.png)

Realizamos el segundo intento de carga agregando el delimiter ';'.

Y como "Id" está en el encabezado y no es de tipo numérico surge el error, con lo cual como no necesitamos el encabezado debemos indicarle que realice la carga sin el header.

![](img_122.png)

Realizamos el tercer intento de carga quitando el header, agregamos "ignoreheader 1" --> ignora la linea 1.

Como vemos hay una tercer falla que indica la existencia de filas en blanco, con lo cual habrá que indicarle que las ignore también.

![](img_123.png)

Realizamos el caurto intento de carga agregando indicando "ignoreblanklines" y finalmente se carga con exito.

```sql

copy estudiante FROM 's3://mibucketredshift/primer_cargue.csv'
credentials 'aws_iam_role=arn:aws:iam::XXXXXX:role/MiRolRedshift'
region 'us-east-2'
delimiter ';'
ignoreheader 1
ignoreblanklines;
```
Pero si observamos los datos faltantes no figuran como valores NULL sino que figuran como valores faltantes o blank (en blanco), esto no es conveniente, con lo cual indicaremos que rellene los valores blank con NULL.

![](img_124.png)

Primero limpiamos la tabla conservando el schema realizando un `TRUNCATE TABLE nombre_tabla` y agregamos el parámetro `blanksasnull`.

```sql 
TRUNCATE TABLE estudiante;

copy estudiante FROM 's3://mibucketredshift/primer_cargue.csv'
credentials 'aws_iam_role=arn:aws:iam::XXXXXX:role/MiRolRedshift'
region 'us-east-2'
delimiter ';'
ignoreheader 1
ignoreblanklines
blanksasnull;
```
Como podemos ver los valores faltanes se encuentran como NULL.

![](img_125.png)

Ahora con la misma sintaxis intentemos cargar el segundo archivo "sugundo_cargue.csv", el cual contiene un formato de fecha diferente, y como vemos a continuación la carga falla por este motivo.

![](img_126.png)

Procedemos a indicarle cual es el formato de fecha utilizado en el archivo de origen (segundo_cargue.csv), agregamos `dateformat = 'mm-dd-yyyy'`, con está instrucción, como vemos a continuación el toma el formato original y lo transforma a yyyy-mm-dd que es el tipo de formato "date" con el cual creamos la tabla.

```sql
copy estudiante FROM 's3://mibucketredshift/segundo_cargue.csv'
credentials 'aws_iam_role=arn:aws:iam::XXXXXX:role/MiRolRedshift'
region 'us-east-2'
delimiter ';'
ignoreheader 1
ignoreblanklines
blanksasnull
dateformat 'mm-dd-yyyy';
```

![](img_127.png)

### <a name="mark_22.0"></a> STL_LOAD_ERRORS
[Index](#index_02)

Muestra los registros de todos los errores de carga de Amazon Redshift.

STL_LOAD_ERRORS presenta un historial de todos los errores de carga de Amazon Redshift. Para obtener una lista completa de posibles errores y explicaciones de cargas, consulte [Referencia de error de carga](https://docs.aws.amazon.com/es_es/redshift/latest/dg/r_Load_Error_Reference.html).

Consulte [STL_LOADERROR_DETAIL](https://docs.aws.amazon.com/es_es/redshift/latest/dg/r_STL_LOADERROR_DETAIL.html) para obtener detalles adicionales, como la fila y la columna de datos exactas en las que se produjo un error de análisis, después de consultar STL_LOAD_ERRORS para encontrar información general sobre el error.

Todos los usuarios pueden acceder a esta vista. Los superusuarios pueden ver todas las filas; los usuarios normales solo pueden ver sus datos. Para obtener más información, consulte [Visibilidad de datos en las tablas y vistas de sistema](https://docs.aws.amazon.com/es_es/redshift/latest/dg/cm_chap_system-tables.html#c_visibility-of-data).

Algunas o todas las columnas de esta tabla también están definidas en la vista de monitoreo de SYS [SYS_LOAD_ERROR_DETAIL](https://docs.aws.amazon.com/es_es/redshift/latest/dg/SYS_LOAD_ERROR_DETAIL.html).

#### Columnas de la tabla

|Nombre de la columna	|Tipo de datos|	Descripción|
|:---|:---:|:---|
|userid	|entero	|ID del usuario que generó la entrada.|
|slice|	entero	|Sector en el que se produjo el error.|
|tbl	|entero	|ID de la tabla.|
|starttime	|timestamp	|La hora de inicio de la carga en UTC.|
|session	|entero|	ID de la sesión que realiza la carga.|
|query	|entero	|ID de la consulta. La columna de consulta puede usarse para combinar otras vistas y tablas del sistema.|
|filename|	character (256)	|Ruta completa hacia el archivo de entrada para la carga.
|line_number|	bigint	|Número de línea en el archivo de carga que tiene el error. Para el comando COPY de JSON, el número de línea de la última línea del objeto JSON que tiene el error.|
|colname	|character (127)	|Campo que tiene el error.|
|type	|character (10)	|Tipo de datos del campo.|
|col_length	|character (10)	|Longitud de la columna, si corresponde. Este campo se rellena cuando el tipo de datos tiene una longitud limitada. Por ejemplo, para una columna con un tipo de datos "character (3)", esta columna tendrá el valor "3".|
|position	|entero	|Posición del error en el campo.|
|raw_line|	character (1024)	|Datos de carga sin formato que tienen el error. Caracteres multibyte en la carga de datos que se reemplazan con un punto.|
|raw_field_value	|char (1024)	|El valor antes del análisis del campo "colname" que induce el error de análisis.|
|err_code	|entero	|Código de error.|
|err_reason|	character (100)	|Explicación del error.|
|is_partial|	entero	|Valor que, si es true (1), indica que el archivo de entrada se divide en rangos durante una operación COPY. Si este valor es false (0), el archivo de entrada no se divide.
|start_offset|	bigint	|Valor que, si el archivo de entrada se divide durante una operación COPY, indica el valor de desplazamiento de la división (en bytes). Si se desconoce el número de línea del archivo, el número de línea es -1. Si el archivo no se divide, este valor es 0.|
|copy_job_id|	bigint	|Identificador del trabajo de copia. Un 0 indica que no hay ningún identificador de trabajo.|

### <a name="mark_22.1"></a> STL_LOAD_COMMITS
[Index](#index_02)

Devuelve información para realizar un seguimiento o solucionar problemas en la carga de datos.

En esta vista, se registra el progreso de cada archivo de datos mientras se carga en una tabla de base de datos (registra las cargas exitosas). 

Todos los usuarios pueden acceder a esta vista. Los superusuarios pueden ver todas las filas; los usuarios normales solo pueden ver sus datos. Para obtener más información, consulte [Visibilidad de datos en las tablas y vistas de sistema](https://docs.aws.amazon.com/es_es/redshift/latest/dg/cm_chap_system-tables.html#c_visibility-of-data).

Algunas o todas las columnas de esta tabla también están definidas en las vistas de monitoreo de SYS [SYS_LOAD_DETAIL](https://docs.aws.amazon.com/es_es/redshift/latest/dg/SYS_LOAD_DETAIL.html) y [SYS_LOAD_HISTORY](https://docs.aws.amazon.com/es_es/redshift/latest/dg/SYS_LOAD_HISTORY.html).

#### Columnas de la tabla

|Nombre de la columna	|Tipo de datos	|Descripción|
|:---|:---:|:---|
|userid	|entero	|ID del usuario que generó la entrada.|
|query	|entero	|ID de la consulta. La columna de consulta puede usarse para combinar otras vistas y tablas del sistema.|
|slice	|entero	|Sector cargado para esta entrada.|
|nombre|	character (256)	|Valor definido por el sistema.|
|filename	|character (256)|	Nombre del archivo que del que se hace un seguimiento.|
|byte_offset	|entero	|Esta información es solo para uso interno.
|lines_scanned	|entero	|Cantidad de líneas examinadas del archivo de carga. Este número puede no coincidir con el número de filas que se cargan efectivamente. Por ejemplo, la carga puede examinar y, al mismo tiempo, tolerar una serie de registros incorrectos, en función de la opción MAXERROR en el comando COPY.|
|errors	|entero|	Esta información es solo para uso interno.|
|curtime	|timestamp	|Hora en que se actualizó por última vez esta entrada.|
|status	|entero	|Esta información es solo para uso interno.|
|file_format	|character(16)	|Formato del archivo que se va a cargar. Los valores posibles son los siguientes: Avro, JSON, ORC, Parquet, Texto|
|is_partial|	entero	|Valor que, si es true (1), indica que el archivo de entrada se divide en rangos durante una operación COPY. Si este valor es false (0), el archivo de entrada no se divide.|
|start_offset	|bigint|	Valor que, si el archivo de entrada se divide durante una operación COPY, indica el valor de desplazamiento de la división (en bytes). Cada división de archivos se registra como un registro independiente con el valor start_offset correspondiente. Si el archivo no se divide, este valor es 0.|
|copy_job_id	|bigint	|Identificador del trabajo de copia. Un 0 indica que no hay ningún identificador de trabajo.|

### Lecturas Recomendadas:

https://docs.aws.amazon.com/es_es/redshift/latest/dg/copy-parameters-data-conversion.html#copy-escape

https://github.com/alarcon7a/redshift_course/tree/master/Copy

https://docs.aws.amazon.com/es_es/redshift/latest/dg/r_STL_LOAD_ERRORS.html

https://docs.aws.amazon.com/es_es/redshift/latest/dg/r_STL_LOAD_COMMITS.html

### <a name="mark_23"></a>Manifiestos y uso de COMPUPDATE para carga con compresión automática

### [Index](#index_02)

Ahora trabajaremos sobre el "tercer_cargue.csv" el cual no contiene delimitadores.

### tercer_cargue.csv

```csv
1Carlos   Alarcon  2706-05-2020
2Andres   Perez    4506-05-2020
3Felipe   Lopez    2006-05-2020
```
Para indicarle cómo tiene que separar la data se utiliza el parámetro `fixedwith` seguido de `columna:cantidad_de_caracteres`.

`fixedwith '0:1,1:9,2:9,3:2,4:10'`, este sería:

    - col 0, 1 caracter.
    
    - col 1, 9 caracteres (también incluye los espacios)
    
    - col 2, 9 caracteres (también incluye los espacios)
    
y así sucesivamente.

```sql
copy estudiante FROM 's3://mibucketredshift/tercer_cargue.csv'
credentials 'aws_iam_role=arn:aws:iam::XXXXXX:role/MiRolRedshift'
region 'us-east-2'
fixedwidth '0:1,1:9,2:9,3:2,4:10'
dateformat 'mm-dd-yyyy';
```

![](img_128.png)

### <a name="mark_23.0"></a>Carga con archivo de manifiesto:
[Index](#index_02)

Para esto necesitamos crear un archivo con la extensión `.manifest`

Contenido en JSON del archivo "tests.manifest", como se observa pueden ser multiples fuentes/URLs, mandatory signfica que debe ser mandatorio que se realice la ingesta. Este archivo debe estar en alguno de mis buckets S3.

```sql
{
    "entries": [
        {"url":"s3://mibucketredshift/cargue_1.csv", "mandatory":true},
        {"url":"s3://mibucketredshift/cargue_2.csv", "mandatory":true}
    ]
}
```

### Observación: "cargue_1.csv" y "cargue_2.csv"  son diferentes a los cargados anteriormente, observar que no figuran los parámetros --> ignoreblanklines, blanksasnull, dateformat 'mm-dd-yyyy'.

Sentencia SQL para realizar la ingesta:

```sql
copy estudiante FROM 's3://mibucketredshift/test.manifest'
credentials 'aws_iam_role=arn:aws:iam::XXXXXX:role/MiRolRedshift'
delimiter ';'
ignoreheader 1
manifest
region 'us-east-2';
```

![](img_129.png)


### <a name="mark_23.1"></a>Comando de compresión COMPUPDATE
[Index](#index_02)

Para esta pruebas se creand 2 tablas diferentes sin compresión (ENCODE raw), "sales_compression_on" y "sales_compression_off"

```sql

CREATE TABLE IF NOT EXISTS public.sales_compression_on
(
	salesid INTEGER NOT NULL  ENCODE raw
	,listid INTEGER NOT NULL  ENCODE raw
	,sellerid INTEGER NOT NULL  ENCODE raw
	,buyerid INTEGER NOT NULL  ENCODE raw
	,eventid INTEGER NOT NULL  ENCODE raw
	,dateid SMALLINT NOT NULL  ENCODE RAW
	,qtysold SMALLINT NOT NULL  ENCODE raw
	,pricepaid NUMERIC(8,2)   ENCODE raw
	,commission NUMERIC(8,2)   ENCODE raw
	,saletime TIMESTAMP WITHOUT TIME ZONE   ENCODE raw
);

CREATE TABLE IF NOT EXISTS public.sales_compression_off
(
	salesid INTEGER NOT NULL  ENCODE raw
	,listid INTEGER NOT NULL  ENCODE raw
	,sellerid INTEGER NOT NULL  ENCODE raw
	,buyerid INTEGER NOT NULL  ENCODE raw
	,eventid INTEGER NOT NULL  ENCODE raw
	,dateid SMALLINT NOT NULL  ENCODE RAW
	,qtysold SMALLINT NOT NULL  ENCODE raw
	,pricepaid NUMERIC(8,2)   ENCODE raw
	,commission NUMERIC(8,2)   ENCODE raw
	,saletime TIMESTAMP WITHOUT TIME ZONE   ENCODE raw
);

```

Al momento de la ingesta de datos utilizo el comando "COMPUPDATE ON" que realiza la siguiente tarea, el comando copy carga parcialmente los datos, luego COMPUPDATE ON analiza cual es la mejor compresión para ese tipo de datos, luego borra la carga parcial y vuelve a crear la tabla con el mejor encodign.

```sql
copy sales_compression_on FROM 's3://mibucketredshift/sales_tab.txt'
credentials 'aws_iam_role=arn:aws:iam::XXXXXX:role/MiRolRedshift'
delimiter '\t' timeformat 'MM/DD/YYYY HH:MI:SS' COMPUPDATE ON region 'us-east-2';

copy sales_compression_off FROM 's3://mibucketredshift/sales_tab.txt'
credentials 'aws_iam_role=arn:aws:iam::XXXXXX:role/MiRolRedshift'
delimiter '\t' timeformat 'MM/DD/YYYY HH:MI:SS' COMPUPDATE OFF region 'us-east-2';

SELECT * 
FROM pg_table_def
WHERE tablename = 'sales_compression_on';

SELECT * 
FROM pg_table_def
WHERE tablename = 'sales_compression_off';

```

Validación:

![](img_130.png)

![](img_131.png)

### Lecturas Recomendadas

https://github.com/alarcon7a/redshift_course/tree/master/Copy/Manifest

### <a name="mark_24"></a>Métodos de carga alternativos al comando copy

### [Index](#index_02)

### <a name="mark_24.0"></a>INSERT por lotes.
[Index](#index_02)

Caso de uso --> No tengo archivos planos, No tengo acceso al bucket, cómo puedo insertar? --> "INSERT" con algunas restricciones.

Cargar data por lotes, cuanto más mejor (NO UN INSERT POR CADA FILA!!!).

![](img_132.png)

### <a name="mark_24.1"></a>bulk insert, datos por lotes
[Index](#index_02)

Esto es a travez de un select poblar otra tabla al momento de su creación.

```sql
CREATE TABLE total_price_by_event AS (
SELECT e.eventname, e.starttime, sum(pricepaid) pricepaid, sum(commission) commission 
FROM sales s 
INNER JOIN event e
ON s.eventid = e.eventid 
GROUP BY e.eventname , e.starttime
);
```

### <a name="mark_24.2"></a>deep copy
[Index](#index_02)

```sql
CREATE TABLE likesales (like sales);

INSERT INTO likesales (SELECT * FROM sales);

DROP TABLE sales;

ALTER TABLE likesales RENAME TO sales;
```

### Lecturas Recomendadas

https://docs.aws.amazon.com/redshift/latest/dg/performing-a-deep-copy.html

### <a name="mark_25"></a>¿Cómo ejecutar sentencias UPDATE y DELETE?

### [Index](#index_02)

Caso de uso: Tengo que actualizar datos erroneos, entonces hay que actualizar o elimnar en filas y columnas especificas para dejar mi db como corresponde, debido a que Redshift no es una RDB esto no es lo más sencillo, a continuación veremos como realizarlo.

Para este ejemplo tenemos la tabla "sales" y la tabla "event", en los cuales la suposición es que, las filas de las columnas "pricepaid" y "commission" pertenecientes a la tabla "sales" para un determinado evento "eventid" tiene un problema y debe ser actualizadas.

Columnas de la tabla "sales":

![](img_133.png)

Primero, supongamos que el problema se presenta en un determinado evento llamado "BECK", con lo cual para obtener el "eventid" conociendo el nombre del evento primero debe consultar la talba "event" de la siguiente forma.

```sql

SELECT * FROM event

WHERE eventname = 'BECK'

```
Como se ve a continuación tenemos mutiples "eventid"

![](img_134.png)

Utilizaremos estos "eventid" como sub consulta para buscar todas las ventas que se encuentran en esa lista de eventos.

```sql
SELECT * FROM sales s 
WHERE eventid in (SELECT eventid FROM event e
WHERE eventname = 'BECK');

```
Ya tengo las filas que quiero actualizar y recordemos las columnas que necesito, "pricepaid" y "commission".

![](img_135png)

A continuación creamos una tabla auxiliar "sales_auxiliar" con solo las columnas necesarias, en donde hacemos una transformación multiplicando por 2 el "pricepaid" y la "commission".

```sql
CREATE TABLE sales_auxiliar AS (
SELECT salesid, pricepaid*2 as pricepaid, commission*2 as commission FROM sales s 
WHERE eventid in (SELECT eventid FROM event e
WHERE eventname = 'BECK'));

```
Entonces sabemos que el UPDATE fila a fila/transacción a transacción, no es bueno para las db columnares. 

Nuestro primer UPDATE sera, actualizar la columna "pricepaid" de la tabla "sales" con los valores de "pricepaid" de la tabla "sales_auxiliar" solo cuando los "salesid" coincidan:

```sql
UPDATE sales
SET pricepaid = sa.pricepaid 
FROM sales_auxiliar sa
WHERE sa.salesid = sales.salesid
```

También podemos, primero borrar los datos especificos que necesitamos modificar antes de la carga modificada.

```sql
DELETE FROM sales
USING sales_auxiliar
WHERE sales_auxiliar.salesid = sales.salesid
```


### Lecturas recomendadas

https://docs.aws.amazon.com/redshift/latest/dg/t_updating-inserting-using-staging-tables-.html

### <a name="mark_26"></a>¿Cómo mantener el desempeño de tu base de datos?

### [Index](#index_02)

### <a name="mark_26.0"></a>Analyze
[Index](#index_02)

Analyze, actualiza las estadísticas y los metadatos de nuestra base de datos, y debido a que Redshift busca la mejor forma de resolver una consulta compleja, y esto lo hace basandose en los metadatos que existen de las tablas, pequeños segmentos de datos que me dicen como se encuentra distribuida en que slice existen, cual es la cantidad de datos que tiene, que columnas se utilizan para hacer filtros o combinaciones, y todo esto para que al momento de ejecutar la consulta sepa cual es el mejor camino para resolverla. Si esos metadatos y estadísticas no se encuentran actualizados el performance de Redshift puede bajar bastante.

![](img_136.png)

Como el comando Analyze consume ciertos recurso, este solo se ejecuta automaticamente cuando la cantidad de datos insertados supera un cierto porcentaje. Pero también lo podemos hacer manualmente.

Acontinuación puedo indicar que se correrá Analyze cuando el porcentaje de cambio sea del 10%.

![](img_137.png)

### <a name="mark_26.1"></a>Vacuum (Limpieza)
[Index](#index_02)

En Redshift cuando eliminamos datos de una tabla, en background estos datos siguen almacenados por un tiempo hasta que se corre la operación de Vacuum.

- Full: Si borre muchos registros de una tabla, los eliminará liberando espacio.

- Sort Only: Supongamos que tengo una tabla, la cual recibirá 10 COPYs (inserto 10 lotes de datos), si esa tabla está ordenada, esos lotes de datos, no se insertaran de manera ordenada al instante, en definitiva, luego de esos 10 COPYs mi tabla se encuentra desordenada en mis Nodos y en mis Slices, SORT ONLY reordena y redistribuye los datos para optimizar el consumo.

![](img_140.png)

![](img_141.png)

Como se ve a continuación la data termina ordenada según el criterio de una llave de ordenamiento, o según el criterio que los Nodos indiquen deba ser ordenada, para finalmente hacer un merge a la tabla original con el ordenamiento necesario.

![](img_142.png)

### Observación: 

Tener en cuenta que Vacuum Sort es una operación costosa en terminos de tiempo, que también indispone la tabla durante el proceso.

- Delete Only: Es la tarea de limpieza del delete.

- Reindex: Es unica y especificamente para tipos de ordenamientos intercalados, reordena en cada Nodo y Slice para ese tipo de tablas con ordenamiento intercalado.

![](img_138.png)

Vacuum también maneja un porcentaje de "desorden", por lo cual si encuetro que la tabla está en cierto porcentaje desordenada se correrá la tarea.

![](img_139.png)

### Lecturas recomendadas:

https://docs.aws.amazon.com/es_es/redshift/latest/dg/r_ANALYZE.html

https://docs.aws.amazon.com/es_es/redshift/latest/dg/r_VACUUM_command.html

### <a name="mark_27"></a>Estadísticas y limpieza de las tablas
### [Index](#index_02)

### <a name="mark_27.0"></a>PG_STATISTIC_INDICATOR
[Index](#index_02)

Almacena información acerca del número de filas insertadas o eliminadas desde la última vez que se ejecutó ANALYZE. La tabla PG_STATISTIC_INDICATOR se actualiza frecuentemente siguiendo las operaciones DML, por lo que las estadísticas son aproximadas.

Solo los superusuarios pueden ver PG_STATISTIC_INDICATOR. Para obtener más información, consulte [Visibilidad de datos en las tablas y vistas de sistema](https://docs.aws.amazon.com/es_es/redshift/latest/dg/cm_chap_system-tables.html#c_visibility-of-data).

#### Columnas de la tabla

|Nombre de la columna|	Tipo de datos	|Descripción|
|:---|:---:|:---|
|stairelid	|oid	|ID de la tabla|
|stairows|	float	|Cantidad total de filas en la tabla.|
|staiins	|float	|Número de filas insertadas desde el último ANALYZE.|
|staidels|	float	|Número de filas eliminadas o actualizadas desde el último ANALYZE.|

#### Observación: (OID) Identificadores Únicos de Objetos, Según definición de la Unión Internacional de Telecomunicaciones (ITU) un OID es: “un valor único global asociado con un objeto que lo identifica sin ambigüedades”.

![](img_143.png)

![](img_144.png)

Puedo hacer el Analyze de diferentes maneras, lo puedo hacer por "columnas", por "columnas de predicado" que son las columnas que usualmente Redshift reconoce que se les hace un "where", o puedo hacer un análisis de estadística total.

Supongamos que sabemos sobre la inserción reciente de datos en las columnas "saleid", y "pricepaid" de la tabla "sales", puedo correr el comando Analyze para esas columnas y tabla, actualizando las estadísticas de "sales" por las columnas indicadas.

```sql
analyze sales(saleid, pricepaid);
```

![](img_145.png)

Realizamos el analyze pero por "columnas de predicado", que son las columnas identificadas por Redshift como columnas que siempre se utilizan para filtrar datos.

```sql
analyze sale predicate columns;
```
![](img_146.png)

Por último le puedo indicar que actualize las estadísticas de toda la tabla "sales".

```sql
analyze sales;
```
![](img_147.png)

### <a name="mark_27.1"></a>STL_ANALYZE
[Index](#index_02)

Log de ejecución de análisis manual o automatico realizado por Redshift.

Registra detalles para las operaciones [ANALYZE](https://docs.aws.amazon.com/es_es/redshift/latest/dg/r_ANALYZE.html).

Solo los superusuarios pueden acceder a esta vista. Para obtener más información, consulte [Visibilidad de datos en las tablas y vistas de sistema](https://docs.aws.amazon.com/es_es/redshift/latest/dg/cm_chap_system-tables.html#c_visibility-of-data).

Algunas o todas las columnas de esta tabla también están definidas en la vista de monitoreo de SYS [SYS_ANALYZE_HISTORY](https://docs.aws.amazon.com/es_es/redshift/latest/dg/SYS_ANALYZE_HISTORY.html).

#### Columnas de la tabla

|Nombre de la columna|	Tipo de datos|	Descripción|
|:---|:---:|:---|
|userid|	entero	|ID del usuario que generó la entrada.|
|xid|	long	|El ID de la transacción.|
|base de datos|	char (30)	|El nombre de la base de datos.|
|table_id	|entero	|El ID de la tabla.|
|status|	char (15)	|El resultado del comando analyze. Los posibles valores son Full, Skipped y PredicateColumn.|
|rows|	double|	La cantidad total de filas en la tabla.|
|modified_rows|	double	|La cantidad total de filas que se modificaron desde la última operación ANALYZE.|
|threshold_percent	|entero	|El valor del parámetro analyze_threshold_percent.|
|is_auto	|char(1)	|El valor es true (t) si la operación incluía una operación analyze de Amazon Redshift de manera predeterminada. El valor es false (f) si el comando ANALYZE se ejecutaba explícitamente.|
|starttime	|timestamp	|La hora en UTC en que comenzó a ejecutarse la operación analyze.|
|endtime|	timestamp	|La hora en UTC en que terminó de ejecutarse la operación analyze.|
|prevtime|	timestamp	|La hora en UTC en que se analizó previamente la tabla.|
|num_predicate_cols|	entero	|La cantidad actual de columnas de predicados en la tabla.|
|num_new_predicate_cols|	entero	|La cantidad de columnas nuevas de predicado desde la operación analyze anterior.|
|is_background	|character (1)	|El valor es true (t) si una operación analyze automática ejecutaba el análisis. De lo contrario, el valor es false (f).|
|auto_analyze_phase	|character (100)	|Se reserva para uso interno.|
|schema_name	|char(128)	|Nombre del esquema para la tabla.|
|table_name|	char (136)	|El nombre de la tabla.|

### <a name="mark_27.2"></a>Vacuum:
[Index](#index_02)

A continuación podemos ver como utilizar la función "svv_table_info" para extraer información de las las columnas "table", unsorted, vacuum_sort_benefit.

En donde unsorted me indica que porcentaje está desordenado, y "vacuum_sort_benefit" cual será el beneficio de realizar la operación de limpieza vacuum sort.

```sql
SELECT "table", unsorted, vacuum_sort_benefit
FROM svv_table_info;

vacuum sales;
vacuum sort only sales to 75 percent;
vacuum delete only sales to 75 percent;
vacuum reindex sales; --Solo funciona se el ordenamiento es intercalado/interleaved
```


### Lectura recomendada:

https://docs.aws.amazon.com/es_es/redshift/latest/dg/r_PG_STATISTIC_INDICATOR.html

https://docs.aws.amazon.com/es_es/redshift/latest/dg/r_STL_ANALYZE.html

### <a name="mark_28"></a>Agrupamiento, ordenamiento y subqueries
### [Index](#index_03)

Pensar todo lo siguiente en un contexto de mejora de performance y no en lógica.

![](img_148.png)

Usar join por llaves --> usar left join, right join, etc si solo utilizar join, esto pasara a un plano cartesiano.

Las funciones funcionan filas por filas, muy costoso.

![](img_149.png)

Si la subconsulta retorna más de 200 registros, mejor utilizar un join.

![](img_150.png)

![](img_151.png)


### <a name="mark_29"></a>¿Qué es y cómo interpretar un explain plan?
### [Index](#index_03)

Cómo mido el performance? --> EXPLAIN

Ejemplo: De las tablas "event" y "venue" quiero las la siguiente consulta, pero ejecuto la consulta con la palabra "explain" delante.

```sql
explain 
select eventid, eventname, event.venueid, venuename
from event, venue;
```

- Como vimos anteriormente el `from vent, venue` ejecutado de esa forma produce un producto cartesiano "Nested Loop", lo cual es muy malo para el performance.

- Luego de ejecutar EXPLAIN se muestran diferentes parámetros, por ejemplo que tipo de query estamos ejecutando, "Nested Loop", seguido a ésta veremos una función, para este caso particular es "DS_BCAST_INNER", las funciones que aparecen en esa posición, nos indican cuan buenas o malas son las queries, a continuación una lista de ellas.

![](img_152.png)

- Luego vemos el costo de la query "cost=0.00..56595631.90 rows=17777196 widht=43", siendo 0.00 cuanto me cuesta traer la primer fila, siendo 56595631.90 cuanto me cuesta traer todas las filas, seguido de rows=17777196 la cantidad de filas que retornará, y finalmente widht=43 cuantas pesa en bytes.

- cost=0.00..56595631.90, es una estimación de cuanto es el costo de aplicar un operador, para este caso Nested Loop, son valores relativos y no tienen una traducción directa, sin embargo pueden ser utilizados para comparar el costo entre diferentes operaciones.

De igual manera puedo interpretar cada una de las ejecuciones.

![](img_153.png)

Cómo cambia el performance si a la misma query le agregamos un filtro where, en donde hacemos coincidir los "venueid"

```sql
explain
select eventid, eventname, event.venueid, venuename
from event, venue
where event.venueid = venue.venueid;
```

Como vemos a continuación ya no tenemos un Nested Loop, sino un Hash Join con un menor costo y menor cantidad de filas.

![](img_154.png)

Finalmente veremos un ejemplo el cual es el cruce óptimo para las tablas de Redshift, para ello primero exploramos las tablas "event" y "venue" con "pg_table_def".

```sql
select * from pg_table_def
where tablename in ('event', 'venue');
```
Podemos ver que la tabla "venue" sí tiene una llave de distribución por "venueid", pero la tabla event no la tiene, con lo cual crearemos una nueva tabla "event_2" para que tenga su llave de distribución y de ordenamiento también por "venueid".

![](img_155.png)

A continuación creo una nueva tabla "event_2" con una llave de distribución y ordenamiento en "venueid".

```sql
CREATE TABLE IF NOT EXISTS public.event_2
(
	eventid INTEGER NOT NULL  ENCODE az64
	,venueid SMALLINT NOT NULL  ENCODE az64 distkey sortkey
	,catid SMALLINT NOT NULL  ENCODE az64
	,dateid SMALLINT NOT NULL  ENCODE RAW
	,eventname VARCHAR(200)   ENCODE lzo
	,starttime TIMESTAMP WITHOUT TIME ZONE   ENCODE az64
);
```
Poblamos "event_2" con los mismos datos de "event"

```sql
insert into event_2 (select * from event);
```

Ahora utilicemos nuestra nueva tabla "event_2" con la misma query.

```sql
explain
select eventid, eventname, event_2.venueid, venuename
from event_2, venue
where event_2.venueid = venue.venueid;
```
Vemos que el costo se ha reducido muchisimo, pero con un tamaño en bites considerablemente mayor, pero como es una tabla recien creada aún no tiene las estadísticas actualizadas, tengamos en cuenta que EXPLAIN utiliza la metada y las estadísticas para evaluar la mejor ejecución, con lo cual hay que correr un analyze.

![](img_156.png)

Corremos el analyze.

```sql
analyze event_2
```
Corremos la query nuevamente.

![](img_157.png)

De mejor a peor.

Merge Join --> Hash Join --> Nested Loop

### Ejemplos utilizando funciones de agregación.

Ejecutamos la siguiente query.

```sql
explain
select e.eventname, sum(pricepaid)
from sales s
inner join event e
where e.eventid = s.eventid
group by e.eventname;
```

![](img_158.png)

![](img_159.png)

A continuación sobre la tabla event probaremos algunas queries.

![](img_160.png)

La misma query pero con un "order by" por una columna que no tiene ordenamiento.

![](img_161.png)

Como vemos a continuación "dateid" es la columna que sí tiene llave de ordenamiento.

![](img_162.png)

La performance mejora muchísimo.

![](img_163.png)

### <a name="mark_29.0"></a>Alerta sobre el performance de las queries con stl_alert_event_log
[Index](#index_03)

![](img_164.png)

### <a name="mark_29.1"></a>STL_QUERY
[Index](#index_03)

Devuelve información acerca de la ejecución de la consulta de una base de datos.

![](img_165.png)

### <a name="mark_29.2"></a>SVL_QLOG
[Index](#index_03)

La vista SVL_QLOG tiene un registro de todas las consultas que se ejecutan para la base de datos.

Amazon Redshift crea la vista SVL_QLOG como un subconjunto legible de información de la tabla STL_QUERY. Utilice esta tabla para encontrar el ID de consulta correspondiente a una consulta recientemente ejecutada o para ver cuánto demora en completarse una consulta.

SVL_QLOG es visible para todos los usuarios. Los superusuarios pueden ver todas las filas; los usuarios normales solo pueden ver sus datos. Para obtener más información, consulte [Visibilidad de datos en las tablas y vistas de sistema](https://docs.aws.amazon.com/es_es/redshift/latest/dg/cm_chap_system-tables.html#c_visibility-of-data).

Algunas o todas las columnas de esta tabla también están definidas en la vista de monitoreo de SYS [SYS_QUERY_HISTORY](https://docs.aws.amazon.com/es_es/redshift/latest/dg/SYS_QUERY_HISTORY.html).

#### Columnas de la tabla

|Nombre de la columna	|Tipo de datos	|Descripción|
|:---|:---:|:---|
|userid|	entero	|ID del usuario que generó la entrada.|
|query	|entero|	ID de la consulta. Puede usar este ID para combinar distintas vistas y tablas del sistema.|
|xid	|bigint	|ID de transacción.|
|pid	|entero	|ID del proceso asociado a la consulta.|
|starttime	|timestamp	|Hora exacta en que la instrucción comenzó a ejecutarse, con seis dígitos de precisión para las fracciones de segundos; por ejemplo: 2009-06-12 11:29:19.131358|
|endtime	|timestamp	|Hora exacta en que la instrucción terminó de ejecutarse, con seis dígitos de precisión para las fracciones de segundos; por ejemplo: 2009-06-12 11:29:19.193640|
|elapsed	|bigint	|Tiempo que demoró la consulta en ejecutarse (en microsegundos).
|aborted	|entero	|Si el sistema detuvo la consulta o el usuario la canceló, esta columna tendrá el valor 1. Si la consulta se ejecutó hasta su finalización, esta columna tendrá el valor 0. Las consultas que se cancelaron por motivos de administración de cargas de trabajo y que se reiniciaron después también tienen un valor 1 en esta columna.|
|label	|character(320)	|Ya sea el nombre del archivo utilizado para ejecutar la consulta o una etiqueta definida con un comando SET QUERY_GROUP. Si la consulta no se basa en archivos o si no se establece el parámetro QUERY_GROUP, el valor del campo es default.|
|subcadena	|character (60)	|Texto de consulta truncado.|
|source_query	|entero	|Si la consulta utilizó el almacenamiento en caché de los resultados, es el ID de la consulta de origen de los resultados almacenados en caché. Si no se utilizó el almacenamiento en caché, el valor de este campo es NULL.|
|concurrency_scaling_status_txt|	texto	|Una descripción que indica si la consulta se ejecutó en el clúster principal o en un clúster de escalado de simultaneidad.|
|from_sp_call	|entero	|Si la consulta se llamó desde un procedimiento almacenado, el ID de la consulta de la llamada del procedimiento. Si la consulta no se ejecutó como parte de un procedimiento almacenado, este campo es NULL.|

### EXPLAIN

Muestra el plan de ejecución de una instrucción de consulta sin ejecutar la consulta. Para obtener información sobre el flujo de trabajo del análisis de consultas, consulte [Flujo de trabajo de análisis de consultas](https://docs.aws.amazon.com/es_es/redshift/latest/dg/c-query-analysis-process.html).

#### Sintaxis


`EXPLAIN [ VERBOSE ] query`
                  
#### Parámetros

_ VERBOSE

Muestra el plan de consulta completo en lugar de solo un resumen.

_ Query

Instrucción de consulta que se explicará. La consulta puede ser una instrucción SELECT, INSERT, CREATE TABLE AS, UPDATE o DELETE.

#### Notas de uso

El rendimiento de EXPLAIN a menudo se ve afectado por el tiempo que lleva crear tablas temporales. Por ejemplo, una consulta que usa la optimización de subexpresión común requiere que se creen y analicen tablas temporales para devolver el resultado de EXPLAIN. El plan de consulta depende del esquema y las estadísticas de las tablas temporales. Por lo tanto, el comando EXPLAIN para este tipo de consulta puede tardar más tiempo en ejecutarse que lo esperado.

#### Puede usar EXPLAIN solo para los siguientes comandos:

SELECT

SELECT INTO

CREATE TABLE AS

INSERT

UPDATE

DELETE

El comando EXPLAIN producirá un error si lo utiliza para otros comandos SQL, como operaciones de base de datos o lenguaje de definición de datos (DDL).

Amazon Redshift utiliza los costos unitarios relativos de los resultados de EXPLAIN para elegir un plan de consultas. Amazon Redshift compara los tamaños de varias estimaciones de recursos para determinar el plan.

#### Pasos de planificación y ejecución de consultas

El plan de ejecución para una instrucción de consulta de Amazon Redshift específica desglosa la ejecución y el cálculo de una consulta en una secuencia discreta de operaciones de tabla y pasos que eventualmente producen un conjunto de resultados finales para la consulta. Para obtener información acerca de la planificación de consultas, consulte [Procesamiento de consultas](https://docs.aws.amazon.com/es_es/redshift/latest/dg/c-query-processing.html).

La siguiente tabla proporciona un resumen de los pasos que Amazon Redshift puede utilizar con objeto de desarrollar un plan de ejecución para cualquier consulta que un usuario envíe para su ejecución.

|Operadores EXPLAIN|	Pasos de ejecución de la consulta|	Descripción|
|:---|:---:|:---|
|SCAN:|
|Sequential Scan|	scan|	Es el paso o el operador de análisis de tabla, o el análisis de la relación de Amazon Redshift. Analiza la tabla completa de manera secuencial de principio a fin y evalúa las restricciones de la consulta para cada una de las filas (Filtro) si se especifica con la cláusula WHERE. También se utiliza para ejecutar las instrucciones INSERT, UPDATE y DELETE.|
|JOINS: Amazon Redshift utiliza diferentes operadores de combinación en función del diseño físico de las tablas que se combinarán, de la ubicación de los datos que se necesitan para la combinación y de los atributos específicos de la propia consulta. Análisis de subconsulta -- El anexo y análisis de subconsultas se utilizan para ejecutar consultas UNION.|
|Nested Loop|	nloop	|Combinación menos óptima; se utiliza principalmente para las combinaciones cruzadas (productos cartesianos; sin una condición de combinación) y algunas combinaciones de desigualdades.|
|Hash Join|	hjoin|	También se utiliza para combinaciones internas y combinaciones externas de izquierda y derecha; por lo general, es más rápida que una combinación de bucle anidado. La combinación hash lee la tabla hash externa, aplica la función hash a la columna de combinación y encuentra coincidencias en la tabla hash interna. El paso puede verterse en el disco. (La entrada interna de hjoin es un paso hash que puede estar basado en el disco).|
|Merge Join	|mjoin	|También se utiliza para combinaciones internas y externas (para tablas de combinación que están distribuidas y ordenadas en las columnas de combinación). Por lo general, es el algoritmo de combinación de Amazon Redshift más rápido, sin incluir otras consideraciones en torno al costo.|
|AGGREGATION: operadores y pasos que se utilizan en las consultas que implican funciones de agregación y operaciones GROUP BY.|
|Aggregate	|aggr|	Operador/paso para las funciones de agregación escalar.|
|HashAggregate	|aggr|	Operador/paso para las funciones de agregación agrupadas. Puede funcionar desde el disco en virtud de la tabla hash que se vierte en el disco.|
|GroupAggregate	|aggr	|En ocasiones, es el operador elegido para consultas de agrupación en conjunto si la configuración de Amazon Redshift del ajuste force_hash_grouping está desactivada.|
|SORT: operadores y pasos que se utilizan cuando las consultas tienen que ordenar o fusionar conjuntos de resultados.|
|Sort	|sort	|Sort realiza la ordenación especificada por la cláusula ORDER BY y, también, otras operaciones como UNION y combinaciones. Puede funcionar desde el disco.|
|Merge|	merge|	Produce los resultados ordenados finales de una consulta en función de los resultados ordenados intermedios que se derivan de las operaciones realizadas en paralelo.|
|Operaciones EXCEPT, INTERSECT y UNION:|
|SetOp Except [Distinct]|	hjoin	|Se usa para consultas EXCEPT. Puede funcionar desde el disco dado que el hash de entrada puede estar basado en el disco.|
|Hash Intersect [Distinct]|	hjoin	|Se usa para consultas INTERSECT. Puede funcionar desde el disco dado que el hash de entrada puede estar basado en el disco.|
|Append [All Distinct]|	save	|Anexo utilizado con el análisis de subconsulta para implementar las consultas UNION y UNION ALL. Puede funcionar desde el disco en virtud del comando "save".|
|Otros:|
|Hash|	hash|	Se utiliza para combinaciones internas y combinaciones externas de izquierda y derecha (proporciona una entrada a la combinación hash). El operador Hash crea la tabla hash para la tabla interna de una combinación. (La tabla interna es la tabla que se revisa para encontrar coincidencias y, en una combinación de dos tablas, es por lo general la más pequeña de las dos).|
|Límite|	limit|	Evalúa la cláusula LIMIT.|
|Materialize|	save|	Materializa las filas para ingresarlas en combinaciones de bucle anidado y en algunas combinaciones de fusión. Puede funcionar desde el disco.|
|--	|parse	|Se usa para analizar datos de entrada de texto durante una carga.|
|--	|project	|Se usa para reordenar columnas y expresiones de computación, es decir, datos del proyecto.|
|Resultado	|--	|Ejecuta funciones escalares que no implican el acceso a ninguna tabla.|
|--|	return	|Devuelve filas al nodo principal o cliente.|
|Subplan	|--	|Se utiliza para algunas subconsultas.|
|Unique	|unique|	Elimina los duplicados de las consultas SELECT DISTINCT y UNION.|
|Window	|window|	Calcula la adición y clasificación de funciones de ventana. Puede funcionar desde el disco.|
|Operaciones de red:|
|Network (Broadcast)	|bcast	|Broadcast también es una atributo de los operadores y pasos de Join Explain.|
|Network (Distribute)	|dist	|Distribuye filas en nodos de computación para el procesamiento paralelo del clúster del data warehouse.|
|Network (Send to Leader)|	return|	Envía los resultados de regreso al nodo principal para seguir trabajando con ellos.|
|Operaciones DML (operadores que modifican datos):|
|Insert (using Result)|	insert|	Inserta datos.|
|Delete (Scan + Filter)	|eliminar	|Elimina datos. Puede funcionar desde el disco.|
|Update (Scan + Filter)	|delete, insert	|Implementado como eliminar e insertar.|

#### Uso de EXPLAIN para RLS

Si una consulta contiene una tabla que está sujeta a políticas de seguridad de la fila (RLS), EXPLAIN muestra un nodo especial SecureScan de RLS. Amazon Redshift también registra el mismo tipo de nodo en la tabla del sistema STL_EXPLAIN. EXPLAIN no revela el predicado de RLS que se aplica a dim_tbl. El tipo de nodo SecureScan de RLS sirve como indicador de que el plan de ejecución contiene operaciones adicionales que son invisibles para el usuario actual.

En el siguiente ejemplo, se ilustra un nodo SecureScan de RLS.


EXPLAIN
SELECT D.cint
FROM fact_tbl F INNER JOIN dim_tbl D ON F.k_dim = D.k
WHERE F.k_dim / 10 > 0;
                               QUERY PLAN
------------------------------------------------------------------------
 XN Hash Join DS_DIST_ALL_NONE  (cost=0.08..0.25 rows=1 width=4)
   Hash Cond: ("outer".k_dim = "inner"."k")
   ->  *XN* *RLS SecureScan f  (cost=0.00..0.14 rows=2 width=4)*
         Filter: ((k_dim / 10) > 0)
   ->  XN Hash  (cost=0.07..0.07 rows=2 width=8)
         ->  XN Seq Scan on dim_tbl d  (cost=0.00..0.07 rows=2 width=8)
               Filter: (("k" / 10) > 0)
Para habilitar una investigación completa de los planes de consultas que están sujetos a RLS, Amazon Redshift ofrece los permisos del sistema EXPLAIN RLS. Los usuarios a los que se les haya otorgado este permiso pueden inspeccionar planes de consulta completos que también incluyan predicados de RLS.

En el siguiente ejemplo, se ilustra un análisis de secuencia adicional debajo del nodo SecureScan de RLS que también incluye el predicado de política de RLS (k_dim > 1).


EXPLAIN SELECT D.cint
FROM fact_tbl F INNER JOIN dim_tbl D ON F.k_dim = D.k
WHERE F.k_dim / 10 > 0;
                                   QUERY PLAN
---------------------------------------------------------------------------------
 XN Hash Join DS_DIST_ALL_NONE  (cost=0.08..0.25 rows=1 width=4)
   Hash Cond: ("outer".k_dim = "inner"."k")
   *->  XN RLS SecureScan f  (cost=0.00..0.14 rows=2 width=4)
         Filter: ((k_dim / 10) > 0)*
         ->  *XN* *Seq Scan on fact_tbl rls_table  (cost=0.00..0.06 rows=5 width=8)
               Filter: (k_dim > 1)*
   ->  XN Hash  (cost=0.07..0.07 rows=2 width=8)
         ->  XN Seq Scan on dim_tbl d  (cost=0.00..0.07 rows=2 width=8)
               Filter: (("k" / 10) > 0)
Mientras se concede el permiso EXPLAIN RLS a un usuario, Amazon Redshift registra el plan de consulta completo, incluidos los predicados de RLS, en la tabla del sistema STL_EXPLAIN. Las consultas que se ejecutan mientras no se concede este permiso se registrarán sin los internos de RLS. La concesión o eliminación del permiso EXPLAIN RLS no cambiará lo que Amazon Redshift haya registrado en STL_EXPLAIN para consultas anteriores.

Relaciones de Redshift protegidas por RLS de AWS Lake Formation
En el siguiente ejemplo se ilustra un nodo LF SecureScan que puede utilizar para ver las relaciones entre Lake Formation y RLS.


EXPLAIN
SELECT *
FROM lf_db.public.t_share
WHERE a > 1;
QUERY PLAN
---------------------------------------------------------------
XN LF SecureScan t_share  (cost=0.00..0.02 rows=2 width=11)
(2 rows)
Ejemplos

nota
Para estos ejemplos, la salida de ejemplo puede variar según la configuración de Amazon Redshift.

El siguiente ejemplo devuelve el plan de consulta para una consulta que selecciona EVENTID, EVENTNAME, VENUEID y VENUENAME de la tablas EVENT y VENUE:


explain
select eventid, eventname, event.venueid, venuename
from event, venue
where event.venueid = venue.venueid;

                                QUERY PLAN
--------------------------------------------------------------------------
XN Hash Join DS_DIST_OUTER  (cost=2.52..58653620.93 rows=8712 width=43)
Hash Cond: ("outer".venueid = "inner".venueid)
->  XN Seq Scan on event  (cost=0.00..87.98 rows=8798 width=23)
->  XN Hash  (cost=2.02..2.02 rows=202 width=22)
->  XN Seq Scan on venue  (cost=0.00..2.02 rows=202 width=22)
(5 rows)
El siguiente ejemplo devuelve el plan de consulta para la misma consulta con resultados más detallados:


explain verbose
select eventid, eventname, event.venueid, venuename
from event, venue
where event.venueid = venue.venueid;

                                QUERY PLAN
--------------------------------------------------------------------------
{HASHJOIN
:startup_cost 2.52
:total_cost 58653620.93
:plan_rows 8712
:plan_width 43
:best_pathkeys <>
:dist_info DS_DIST_OUTER
:dist_info.dist_keys (
TARGETENTRY
{
VAR
:varno 2
:varattno 1
...

XN Hash Join DS_DIST_OUTER  (cost=2.52..58653620.93 rows=8712 width=43)
Hash Cond: ("outer".venueid = "inner".venueid)
->  XN Seq Scan on event  (cost=0.00..87.98 rows=8798 width=23)
->  XN Hash  (cost=2.02..2.02 rows=202 width=22)
->  XN Seq Scan on venue  (cost=0.00..2.02 rows=202 width=22)
(519 rows)
El siguiente ejemplo devuelve el plan de consulta para una instrucción CREATE TABLE AS (CTAS):


explain create table venue_nonulls as
select * from venue
where venueseats is not null;

QUERY PLAN
-----------------------------------------------------------
XN Seq Scan on venue  (cost=0.00..2.02 rows=187 width=45)
Filter: (venueseats IS NOT NULL)
(2 rows)



### Lecturas recomendadas

https://docs.aws.amazon.com/es_es/redshift/latest/dg/c_data_redistribution.html

https://docs.aws.amazon.com/es_es/redshift/latest/dg/r_STL_ALERT_EVENT_LOG.html

https://docs.aws.amazon.com/es_es/redshift/latest/dg/r_STL_QUERY.html

https://docs.aws.amazon.com/es_es/redshift/latest/dg/r_SVL_QLOG.html

### <a name="mark_30"></a>¿Cómo descargar datos eficientemente con UNLOAD?
### [Index](#index_03)

Supongamos que tenemos toda nuestra data en Redshift y alguien en nuestro equipo no tiene acceso a el, entonces cual es la mejor manera de bajar esa información?

Esto se trata de pasar data de mi Redshift a por ejemplo mi S3, lo cual hacemos de la siguiente forma.

```sql
unload ('select * from nombre_tabla')
to 's3://nombre_bucket_s3/unload/unload_test_'
credentials 'aws_iam_role=rol_arn_iam';
```
A continuación en nuestro S3 podemos ver dentro de la carpeta "unlaod" 4 archivos con el prefijo "unload_test_" con el mismo tamaño cada uno.

Fue dividido en 4 archivos porque tengo 2 nodos con 2 slices por nodo. 

![](img_166.png)

Si lo queremos en un solo archivo (no superior a 6.2 GB), NO recomendado!!! agregamos el parámetro "parallel off"

```sql
unload ('select * from nombre_tabla')
to 's3://nombre_bucket_s3/unload/unload_test_'
credentials 'aws_iam_role=rol_arn_iam'
parallel off;
```

### Mismo comando con multiples parámetros

```sql
unload ('select * from unload_test_2')
to 's3://mybucketredshiftsantiago/unload/unload_test_4'
credentials 'aws_iam_role=arn:aws:iam::118590468211:role/MiRoleRedshift'
ALLOWOVERWRITE
delimiter ';'
header
maxfilesize 500 mb
ZSTD --tipo de compresión
manifest
partition by (c_nation) INCLUDE 
;
```
### Lecturas recomendadas:

https://docs.aws.amazon.com/es_es/redshift/latest/dg/r_UNLOAD.html#unload-usage-notes

https://docs.aws.amazon.com/es_es/redshift/latest/dg/r_UNLOAD.html

### <a name="mark_31"></a>Otras tablas útiles de Redshift para entender el comportamiento de nuestros datos
### [Index](#index_03)

### <a name="mark_31.0"></a> SVL_USER_INFO

Puede recuperar datos sobre los usuarios de la base de datos de Amazon Redshift con la vista SVL_USER_INFO.

#### Columnas de la tabla

|Nombre de la columna	|Tipo de datos	|Descripción|
|:---|:---:|:---|
|usename	|texto	|Nombre de usuario de la función.|
|usesysid	|entero	|ID del usuario.|
|usecreatedb	|booleano	|Valor que indica si el usuario tiene permisos para crear bases de datos.|
|usesuper	|booleano	|Valor que indica si el usuario es un superusuario.|
|usecatupd	|booleano	|Valor que indica si el usuario puede actualizar los catálogos de sistema.|
|useconnlimit	|texto	|Número de conexiones que el usuario puede abrir.|
|syslogaccess	|texto	|Valor que indica si el usuario tiene acceso a los registros del sistema. Los dos valores posibles son RESTRICTED y UNRESTRICTED. RESTRICTED significa que los usuarios que no son superusuarios pueden ver sus propios registros. UNRESTRICTED significa que los usuarios que no son superusuarios pueden ver todos los registros de las vistas y tablas del sistema en las que tienen privilegios SELECT.|
|last_ddl_ts|	timestamp	|Marca temporal de la instrucción de creación del lenguaje de definición de datos (DDL) ejecutada por el usuario.|
|sessiontimeout	|entero	|El tiempo máximo en segundos durante el cual una sesión permanece inactiva o en reposo antes de que se cierre. El 0 indica que no se establece tiempo de espera. Para obtener información acerca de la configuración de tiempo de espera en reposo o inactivo del clúster, consulte Cuotas y límites de Amazon Redshift en la Guía de administración de Amazon Redshift.|
|external_id	|texto	|Identificador único del usuario en el proveedor de identidades de terceros.|

### Lecturas recomendadas

https://docs.aws.amazon.com/es_es/redshift/latest/dg/cm_chap_system-tables.html

https://docs.aws.amazon.com/es_es/redshift/latest/dg/r_SVL_USER_INFO.html